# 深度学习知识总结
先从伯禹的学习资料入手，边学习，边 [整理记录](https://github.com/shiyutang/Hands-on-deep-learning) +《
[神经网络和深度学习](https://nndl.github.io/nndl-book.pdf)
》补充；
累了可以听听 [MIT 的6.S191](https://www.youtube.com/watch?v=JN6H4rQvwgY) refresh一下

根据下列框架进行结构性的知识总结。

1.浅层神经网络及模型基础

* [1.1 线性回归](#线性回归)
    * 定义
    * 实施步骤
    * 要点
    * 主要函数
    * 主要问题：
        * 1.构建一个深度学习网络并训练需要哪些步骤
        * 2.什么时候该用parameter.data

* [1.2 分类模型和softmax](#分类模型和softmax)
    * 定义
    * softmax的性质
    * softmax的优势
    * 分类模型
    * 交叉熵函数

* [1.3 多层感知机](#多层感知机)
    * 定义
    * 激活函数
    * 主要问题：
        * 如何选择不同激活函数

* [1.4 模型选择（过拟合欠拟合的出现和解决)](#模型选择（过拟合欠拟合的出现和解决）)
    * 模型选择的方法
    * 欠拟合和过拟合定义和影响因素
    * 欠拟合和过拟合的解决方法
        * 权重衰减和正则化
        * dropout

* [1.5 数值稳定性与模型初始化](#数值稳定性与模型初始化)
    * 梯度消失和梯度爆炸
    * 导致梯度消失和爆炸的原因
    * 神经原初始化

2.卷积神经网络详解
* [2.1 卷积神经网络](#卷积神经网络)
    * 起源和特点
    * 卷积神经网络组成
    * 卷积层及其可选操作
        *  空洞卷积  **todo**
        *  感受野的计算  **todo**
    * Pooling层
    * 归一化层： 
        *  实例归一化 **todo**
        *  批归一化 
        *  组归一化 **todo**
    * 损失函数  **todo**
        *  交叉熵损失函数
        *  L2损失
        *  L1损失
    * 卷积神经网络的整体结构
    * 主要网络架构及其特点
        * Lenet
        * Alexnet
        * VGG
        * Network in Network（NIN）
        * GoogLenet  
        * Resnet  
        * DenseNet

* 3.循环神经网络  **todo**
    * 基础
    * GRU
    * Lstm
    * 深度循环神经网络
    * 双向循环神经网络

* 4.注意力机制  **todo**
    * 基础
    * Seq2seq中的应用

* 5.Transformer  **todo**

* [6.优化](#优化)
    * 深度学习优化：
        * 深度学习优化和普通优化的差异
        * 基于梯度的优化方法的挑战
    * 凸优化
        * 凸性
        * 凸函数的性质和 Jensen 不等式
        * 如何优化带有限制条件的函数 
    * 优化算法
        * 牛顿法
        * 共轭梯度法
        * 随机梯度下降法
        * 小批量随机梯度下降法 (SGD)
    * 高阶优化算法
        * momentum
        * AdaGrad       
        * RMSProp
        * AdaDelta
        * Adam

* 7.数据增强  **todo**

* 8.模型微调  **todo**

* 9.GAN     **todo**
    * basic
    * DCGAN

* 10.目标检测  **todo**

* 11.语义分割  **todo**

* 12.领域自适应  **todo**

* 13.风格迁移  **todo**

* 14.变化检测  **todo**

# 线性回归

__定义__：基于特征和标签之间的线性函数关系约束，线性回归通过建立单层神经网络，将神经网络中每一个神经元当成是函数关系中的一个参数，通过利用初始输出和目标输出建立损失，并优化损失最小的方式使得神经元的数值和真实函数参数数值最相近，从而通过网络训练得到最符合数据分布的函数关系。

__实施步骤__：
1. 初始通过随机化线性函数的参数，通过输入的x，会得到一系列y_h
2. 输出的y_h和真实值y之间因为神经元参数不正确产生差距，为了y_h和y能尽量地逼近，我们通过平方误差损失函数（MSE Loss）来描述这种误差。
3. 类似于通过求导得到损失函数最优解的方式，我们通过梯度下降法将这种误差传递到参数，通过调整参数使误差达到最小
4. 通过几轮的训练，我们得到的最小的损失值对应的神经元数值，就是描述输入输出的线性关系的最好的参数。

__要点__：
1. 确定输入输出之间一定满足线性关系，这一点可以通过对x,y画图看出，只有线性关系才能使用线性回归训练得到
2. 由于线性关系唯一由神经元个数决定，不同的参数个数唯一决定了这种线性关系，因此需要选择适合的特征用于线性回归

## 这一节中出现的有用的函数
1. 使用plt绘制散点图

In [ ]:
from matplotlib import pyplot as plt
plt.scatter(features[:, 1].numpy(), labels.numpy(), 1)

2. 自行制作dataLoader: dataloader 为输入dataset可以随机获取dataset中batch size 个样本
> 通过使用打乱的indices，每次yield batch size个样本，生成的生成器可以用for调用

In [ ]:
import torch
import  random

def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    random.shuffle(indices)  # random read 10 samples
    for i in range(0, num_examples, batch_size):
        j = torch.LongTensor(indices[i: min(i + batch_size, num_examples)])
        # the last time may be not enough for a whole batch
        yield  features.index_select(0, j), labels.index_select(0, j)

3. 参数初始化：自行初始化网络中的参数，使用init模块

In [ ]:
from torch.nn import init

init.normal_(net[0].weight, mean=0.0, std=0.01)
init.constant_(net[0].bias, val=0.0)

## 重要的问题：
### 1.构建一个深度学习网络并训练需要哪些步骤？

深度学习网络的主要组成部分就是数据，网络和训练，因此可以根据这三部分展开为下面几个步骤：

0.数据部分
1. 生成数据集/找到现有数据集
2. 根据数据集构建Dataset 并用之构建dataloader
3. （可选）调用构建的Dataloader，得到数据并可视化，检查实现的正确性，并对数据有一定了解

1.网络部分
4. 定义模型，初始化模型参数
5. 定义损失函数，如本节的MSE loss
6. 定义优化函数，SGD,Adam... 及其参数：学习率，动量，weight_decay...

2.训练部分
7. 使用循环的方式，每个循环训练一遍所有数据
8. 将数据输入网络，根据损失函数和网络输出建立损失
9. 梯度清零，损失回传，优化器更新损失
10. 记录损失，可视化结果，往复训练

### 2.什么时候该用parameter.data?
下面是课程中使用的优化器的代码，可以发现，参数的更新使用了param.data

In [ ]:
def sgd(params, lr, batch_size):
    for param in params:
        param.data -= lr * param.grad / batch_size # ues .data to operate param without gradient track

根据我的理解，这是由于反向传播机制在需要更新的参数进行运算时会构建动态运算图，如果直接使用这个`param`进行更新，就会在动态图中计入这一部分，从而反向传播时也会将这一步运算的梯度加入。而我们实际希望的则是损失函数对参数进行求导，而不希望再此参数上“节外生枝”。因此，在网络前向传播和损失函数计算之外的参数运算，应当使用param.data进行更新



# 分类模型和softmax
_定义_：softmax是用于将向量输出统一为有概率性质的一个函数，基于其指数的性质，它可以拉大大数和小数之间的差距，并且越大的数，为了达到和较小的数的差距只需要更小的差距，有助于筛选出一组向量中的最大值，并使其和其他数更加明显区分。

_性质_: softmax函数的常数不变性，即 softmax(x)=softmax(x+c)：对需要进行softmax变换的向量，同时减去一个较大的数不会对结果产生影响，推导如下：


<div align=center>

![alt text](Pics/softmax.png)

</div>


优势：1. 通过softmax，输入向量每个元素的大小一目了然，不需要关心输出的具体数值，也不会存在输出相差较大时，不好比较的问题
2. 使用softmax的输出进行交叉熵函数的比较相当于统一了量纲，平衡了不同输出结果的重要程度

## 分类模型
_定义_：分类模型用于将输入的图片/信号进行分类，本单元讲到的分类模型是对输入进行一次线性变换，最后将分类结果利用softmax进行区分，示例如下图：


<div align=center>

![Image Name](Pics/fc.png)

</div>

_交叉熵函数_：上节的回归问题，我们用到了MSE，因为我们希望线性模型的参数和真实的完全一致，但是在分类模型中，我们只希望真实类别对应的概率越大越好，其他概率的具体数值对损失并没有过多含义。因此，这里使用了交叉熵函数，通过将标签化为one-hot编码，使得损失函数只关注正确类别对应的概率大小,其函数表达和实例操作如下：
<div align=center>

![Image Name](Pics/cross_ent.png)

</div>

如果标签为one-hot编码且只有一个正确类别，那么最小化交叉熵损失就是最大化正确类别对应输出概率的log形式，从另一个角度来看，最小化交叉熵损失函数等价于最大化训练数据集所有标签类别的联合预测概率


# 多层感知机
_定义_：多层感知机是采用多个神经网络层，并在其中添加了非线性单元，从而使得网络可以得到从输入到输出的非线性映射，从而将线性模型转化为非线性模型，在此基础上，增加网络的深度可以进一步增加网络的复杂度。

## 激活函数：
_定义_：拥有非线性部分的函数，即函数的导数并非处处一致，常见的激活函数有Relu，Leaky-relu，tanh，sigmoid等

实例：


<div align=center>

![Untitled/Untitled.png](Pics/activation.png)

</div>


## 重要的问题：
### 1. 不同的激活函数应当如何选择？
在考虑不同激活函数的区别和优势时，我们主要考虑激活函数对神经元输出的影响和神经元的数值更新的能力。（即神经元的存在以及学习能力）

*神经元的存在问题*：当一个神经元更新到无论什么参数输入，其输出均为零时，其就失活了。这个主要是因为某些激活函数的截断作用，当大量神经元失活，网络的复杂度就会受到严重影响

*神经元参数的更新*：其取决于反向传播到该神经元的梯度值，这个梯度值是累乘的，其会乘上激活函数的梯度。因此，如果激活函数的梯度过小，那么就会导致这个神经元学习缓慢/无法学习


* Relu：由于其计算效率和不容易梯度消失，从而大多数情况下使用；但是，由于其输出范围不确定，因此只能在隐藏层中使用；同时Relu函数因为在小于0的部分为0，因此容易在学习率大时使神经元失活

* Sigmoid函数：用于分类器时，通常效果更好，但是存在由于梯度消失。

* Tanh 函数: 和sigmoid 相似，但是其导数输出在（0，1），相比sigmoid更容易激活神经元；也存在梯度消失的问题


# 模型选择（过拟合欠拟合的出现和解决）
我们在训练深度学习模型时，期望使用训练数据训练的这个模型能在测试数据上也能得到很好的表现，这个就是期望泛化误差减小。（_泛化误差:_ 模型在任意测试数据上的输出结果和真实结果的差距的期望，通常使用测试集上的误差来代表）

为了得到这样一个模型，其中的主要影响因素是模型复杂度，输入数据量，训练时间，训练方法等。但是由于我们没有确切的方式定义这些因素是如何影响的，因此，我们需要各种各样的方法来选择模型。

## 模型选择的方法
选择模型时，我们主要参考两个指标。一个是训练误差，一个是验证误差，这里不能使用泛化误差是因为这样会导致泛化误差有偏，不能反映真实的泛化误差。
训练误差和验证误差分别是模型在训练数据集和验证数据集上的误差，这个误差可以用交叉熵损失函数，MSE损失函数等多种损失函数计算得出
* 验证数据集进行验证：
验证集通过在整体数据中抽出一定比例数据组成，其不参与模型的训练过程，在模型使用训练数据训练完成后，我们通过其在验证集上的精度进行模型的选择。
* k折验证法：
有时，我们的数据不是非常充足，因此为了减少验证集对数据的浪费，我们采用K折验证法：
   1. 我们将整个数据分成k份，其中（K-1）份代表训练集，1份代表验证集，同时我们要遍历所有的组合方式，也就生成K份不同的数据集
   2. 模型分别基于K份数据集中的训练集进行训练，对应的验证集进行验证，获得的验证精度进行平均即获得这个模型的精度。

## 训练模型的过拟合和欠拟合
训练出来的模型一般存在两类问题，导致最终的验证结果不是最理想，因此我们需要尽量避免。
<div align=center>

![Image Name](Pics/OVERFIT.png)
</div>

_欠拟合_：训练误差无法达到较小，说明此时在训练数据上模型都不能正确拟合。
_过拟合_：当训练误差已经较小，但是验证误差和训练误差相差较大，说明模型仅仅拟合到训练数据，该模型并不能泛化。

影响欠拟合和过拟合的因素：
影响模型有没有拟合/过拟合的因素是多方面的，包括模型模型复杂度，数据量，训练时间，训练方法等。
* 模型复杂度：模型复杂度可以受到参数量，网络的广度深度的影响，其最终等效为这个模型能描述的所有的函数的集合。一个过大的模型在一定时间的训练下很容易过度拟合
* 数据量： 数据量不仅仅是数据的多少，其还应该具有足够代表性和多样化。
* 训练时间&训练方法：合适的训练方法和恰当的训练时间可以避免模型过拟合/欠拟合


- 过拟合和欠拟合的解决方法
为了解决过拟合/欠拟合，我们需要对模型的复杂度进行限制/增加更多的数据，下面提出两种解决过拟合的方法
* 权重衰减和正则化：
    在使用SGD优化方法时，正则化和权重衰减具有一致性。其中L2正则化是在模型的损失上添加一个权重的二范数作为惩罚项，从而限制参数的大小。
    具体表达式以线性回归模型的损失函数为例：
    <div align=center>

    ![Untitled/Untitled.png](Pics/linearloss.png)

    </div>

    正则项则是参数的二范数:
    <div align=center>

    ![Untitled/Untitled.png](Pics/loss_reg.png)

    </div>

    最终，根据SGD的参数更新方式，我们可以得到如下更新公式：
    <div align=center>

    ![Untitled/Untitled.png](Pics/SGDupdate.png)

    </div>

    可以发现，添加了L2正则项的损失函数促使更新公式在权重项之前乘以了一个小于1的系数，也就是等效于权重衰减设置为lr*lambda

* inverted dropout 丢弃法：

   dropout的思想是在前向传播的时候，设置一概率p，使得每个神经元有p的概率输出为0，很明显，这样的方式可以达到网络稀疏化的效果，从而削弱了网络的表达能力，即复杂度。从而是一种有效的防止过拟合的方式，下面我们看看其具体是怎么运行的：

    1. 在前向传播的时候，设置一个存在权重n，其有p的概率为0，1-p的概率为1，从而一个神经元输出为：

       这里我们发现其输出的期望还是等于自身：

       也就是增加了dropout的网络的输出平均值与没有增加dropout一致，而我们又知道dropout每次代表不同的网络结构，因此对增加了dropout的网络输出求平均即为对不同网络结构求平均，这样我们实现了一个网络结构和一次训练，但是求出的平均值等效于同时训练多个网络得到的平均值。这一种方法也有效地减少了过拟合。

    2. 在训练过程中我们不断使用不同的数据重复上述步骤

    3. 我们在测试是去除缩放因子，并对网络进行测试，由于第一步我们知道添加dropout与否的网络的平均值相同，那么我们在测试时去除缩放因子就保证了和训练的平均值相同。即我们测试了我们训练的网络。

   那么dropout在网络结构层次上又是怎么影响的呢？
   其作者认为，在训练神经网络时，过拟合现象起源于神经元之间的co-adpatation 相互适应，也就是某个神经元在给定某些固定特征时，其强行根据一些不正确的特征组合出最终的正确答案，因此dropout就打破了特征和输出的固定联系，从而促使网络根据一个更加general的场景做决定。

# 数值稳定性与模型初始化
通常我们希望我们的模型在训练过程中能快速地学习并在学习之后收敛到一个较好的结果。因此网络的数值稳定性就格外重要，我们希望网络中的神经元能每次更新适当大小的梯度作为优化方向。而梯度消失和梯度爆炸则会导致上述训练停滞或不能收敛。
> 深度模型有关数值稳定性的典型问题是衰减和爆炸。当神经网络的层数较多时，模型的数值稳定性容易变差。

_梯度消失_：其指的是传递到很多神经元的梯度几乎为0，从而导致网络的学习停滞

_梯度爆炸_：对应于梯度消失，梯度爆炸则代表神经元的梯度过大，使得其一下更新到一个不合适的位置，从而网络不能收敛

## 导致梯度消失和梯度爆炸的原因：
1. 神经网络没有很好地初始化：当我们假设神经网络没有激活层时，传递到某一个神经元的梯度就是其后方连接上所有神经元参数的连乘积。那么，在较深的网络中如果我们将神经元均初始化较大（>1）的数值会导致梯度爆炸，较小（<1）则会导致梯度消失
那么有激活函数的场景怎么样呢，有激活函数时，不同的损失函数的情况有所不同，对于Sigmoid激活函数，当参数初始化较小的值，神经元的输入会较小，此时激活函数对应的梯度也较小，由此影响到神经元的梯度也较小，从而造成了梯度消失。
2. 没有选择恰当的激活函数：如第一点所述，不同的激活函数会对梯度消失和梯度爆炸产生影响，因此需要根据数据选择合理的激活函数
3. 输入没有合理地归一化：影响激活函数梯度的除了其自身之外，还有其输入，即便神经元初始正常，没有归一化的输入也会导致神经元输出过大/小，从而激活函数梯度过小

## 神经元初始化的方法--缓解梯度消失和爆炸
根据上述阐述，我们知道神经元的权重应当初始化到一个1附近的数值/0附近（对于Sigmoid等激活函数来说），因此有各种各样的初始化方法，在pytorch里面可以通过torch.init调用
Xavier初始化：将权重参数初始化到：
<div align=center>

![Untitled/Untitled.png](Pics/Uniform.png)

</div>

根据均匀分布的均值和方差公式，我们可以知道其初始化后，均值为0，方差为2/（a+b）


# 卷积神经网络
上面我们介绍的线性回归和多层感知机等均为全连接网络，但是全连接网络的两个缺点却对图片一类的数据很不友好：
1. 参数量过大： 全连接层在处理输入（h1,w1,c1）输出（h2,w2,c2）时， 需要的参数量为h1\*w1\*c1\*h2\*w2\*c2, 对于200\*200\*3大小的图片，单层的参数量就达到了14400000000。
2. 没有办法识别空间局部特征，图片往往有空间相对信息，因此上下左右相邻的像素组合比展平的元素更有意义，同时缩放，平移，旋转不会改变这个区域的特征，但是全连接层则需要单独检测。

为了解决上述问题，同时基于生物感受野的思想，卷积神经网络横空出世。
在卷积神经网络中，影响输出元素 x 的前向计算的所有可能输入区域（可能大于输入的实际尺寸）叫做 x 的感受野（receptive field）

通过局部连接，权重共享，汇聚的结构，卷积神经网络解决了上述两个问题并不断发展，下面我们来讲述一下卷积神经网络的组成和技术细节。

## 卷积神经网络组成
卷积神经网络主要有卷积层，池化层，归一化层，全连接层组成。

### 卷积层
首先卷积层中最重要的就是卷积，但是实际上卷积层是做的互相关运算，而不是卷积运算。基于二者是旋转180度的关系，同时卷积层的参数是可学习的，因此我们可以直接使用互相关运算等效替代卷积运算并减少计算量。

卷积运算公式：
<div align=center>

![Untitled/Untitled.png](Pics/conv.png)

</div>
互相关运算公式：
<div align=center>

![Untitled/Untitled.png](Pics/corr.png)

</div>

在对图像做完卷积之后，我们还需要增加一个偏置b到最后的运算结果上，从而和之前的全连接层保持一致，记录与特征无关的平移，如下所示：

<div align=center>

![Untitled/Untitled.png](Pics/convmulc.png)

</div>

<div align=center>

![Untitled/Untitled.png](Pics/convbias.png)

</div>


卷积中的可选操作

在卷积层中除了进行卷积的操作，还可以进行一些其他的操作保证更好地利用原图/特征图上的信息。
* padding：为了保证原图/特征图的边缘也能多次参与和卷积核的运算并保证输入输出大小的不变性，人们采取将周边铺上0元素的方法，我们记单边增加了p列0元素。
* 步长：是指卷积核在滑动时的时间间隔。有时图片的尺寸较大，而我们为了减少计算量，并尽可能多覆盖相同多的特征，就会采用带步长的卷积，我们记步长为s
* 输入输出尺寸对应：设输入单边长度为lin，那么对应的输出单边长度为lout，而卷积核的单边长度为k
<div align=center>

![Untitled/Untitled.png](Pics/convsize.gif)

</div>

除此之外，还有空洞卷积，可变形卷积等其他卷积的变种可以用来进行卷积的操作。

### Pooling层
汇聚层的作用是进行特征选择，降低特征数量，从而减少参数数量，同时可以对一些小的局部形态改变保持不变性，并拥有更大的感受野。但是过大的汇聚层会造成信息的损失。

汇聚层的反向传播：和卷积层的反向传播不同，汇聚层的输入和输出不一致，因此，我们反向传播时需要考虑怎样将误差值从输出传播到输入。

如果下采样是最大汇聚，误差项 𝛿(𝑙+1,𝑝) 中每个值会直接传递到上一层对应区域中的最大值所对应的神经元，该区域中其他神经元的误差项都设为0．如果下采样是平均汇聚，误差项𝛿(𝑙+1,𝑝) 中每个值会被平均分配到上一层对应区域中的所有神经元上
<div align=center>

![Untitled/Untitled.png](Pics/poolgrad.png)

</div>

### 归一化层
通常，为了统一输入数据的量纲，我们会将输入数据进行归一化，从而可以避免量纲这种无关条件对神经网络学习的影响。
但是光对输入进行归一化是不够的，在神经网络学习过程中，内部每一层输出也会不断发生变化，如果限制变化范围，就会使得后层神经层需要不断适应浅层神经层的输出，从而带来训练不稳定和训练速度慢的问题。为了解决这个问题，我们在神经网络的输出之后引入各种类型的归一化操作。

**归一化层的作用就是通过规范每一层神经网络输出的分布，限制前层输出分布变化带来的后层输入的不稳定，从而引起的链式反应。**

#### 批归一化层
我们通常认为卷积层输出某一个通道代表了一类特征，因此批归一化通过将一个通道的所有特征图，使用样本统计量进行归一化并再一次进行仿射变换，获得变换后输出。也就是说，对于（N, C, H, W）的输入，其均值和方差的形状均为（1, C, 1, 1）。批量归一化主要有以下优势：

    1. 在使用sigmoid/tanh激活函数时，限制神经元的输出范围，使得输出落在激活函数梯度较大的区域，避免梯度消失的情况，同样可以减少梯度爆炸的情况
    2. 在深层网络中限制浅层神经网络输出的变化范围，避免深层神经元对变化大的浅层神经元输出的不断适应导致的训练不稳定问题
    4. 起到正则化的效果，因为限制输出的范围，从而将输出空间大大减小，


其推导公式如下：

#### 1.对全连接层做批量归一化
位置：全连接层中的仿射变换和激活函数之间。  
**全连接：**  
$$
\boldsymbol{x} = \boldsymbol{W\boldsymbol{u} + \boldsymbol{b}} \\
 output =\phi(\boldsymbol{x})
 $$   


**批量归一化：**
$$ 
output=\phi(\text{BN}(\boldsymbol{x}))$$


$$
\boldsymbol{y}^{(i)} = \text{BN}(\boldsymbol{x}^{(i)})
$$


$$
\boldsymbol{\mu}_\mathcal{B} \leftarrow \frac{1}{m}\sum_{i = 1}^{m} \boldsymbol{x}^{(i)},
$$ 
$$
\boldsymbol{\sigma}_\mathcal{B}^2 \leftarrow \frac{1}{m} \sum_{i=1}^{m}(\boldsymbol{x}^{(i)} - \boldsymbol{\mu}_\mathcal{B})^2,
$$


$$
\hat{\boldsymbol{x}}^{(i)} \leftarrow \frac{\boldsymbol{x}^{(i)} - \boldsymbol{\mu}_\mathcal{B}}{\sqrt{\boldsymbol{\sigma}_\mathcal{B}^2 + \epsilon}},
$$

这⾥ϵ > 0是个很小的常数，保证分母大于0


$$
{\boldsymbol{y}}^{(i)} \leftarrow \boldsymbol{\gamma} \odot
\hat{\boldsymbol{x}}^{(i)} + \boldsymbol{\beta}.
$$


引入可学习参数：拉伸参数γ和偏移参数β。若$\boldsymbol{\gamma} = \sqrt{\boldsymbol{\sigma}_\mathcal{B}^2 + \epsilon}$和$\boldsymbol{\beta} = \boldsymbol{\mu}_\mathcal{B}$，批量归一化无效。

### 2.对卷积层做批量归⼀化
位置：卷积计算之后、应⽤激活函数之前。  
如果卷积计算输出多个通道，我们需要对这些通道的输出分别做批量归一化，且每个通道都拥有独立的拉伸和偏移参数。
计算：对单通道，batchsize=m,对该通道中m×w×h个元素同时做批量归一化,使用相同的均值和方差。

### 3.预测时的批量归⼀化
训练：以batch为单位,对每个batch计算均值和方差。  
预测：用移动平均估算整个训练数据集的样本均值和方差作为测试时的均值和方差。

### 从零实现

In [ ]:
import time
import torch
from torch import nn, optim
import torch.nn.functional as F
import torchvision
import sys
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def batch_norm(is_training, X, gamma, beta, moving_mean, moving_var, eps, momentum):
    # 判断当前模式是训练模式还是预测模式
    if not is_training:
        # 如果是在预测模式下，直接使用传入的移动平均所得的均值和方差
        X_hat = (X - moving_mean) / torch.sqrt(moving_var + eps)
    else:
        assert len(X.shape) in (2, 4)
        if len(X.shape) == 2:
            # 使用全连接层的情况，计算特征维上的均值和方差
            mean = X.mean(dim=0)
            var = ((X - mean) ** 2).mean(dim=0)
        else:
            # 使用二维卷积层的情况，计算通道维上（axis=1）的均值和方差。这里我们需要保持
            # X的形状以便后面可以做广播运算
            mean = X.mean(dim=0, keepdim=True).mean(dim=2, keepdim=True).mean(dim=3, keepdim=True)
            var = ((X - mean) ** 2).mean(dim=0, keepdim=True).mean(dim=2, keepdim=True).mean(dim=3, keepdim=True)
        # 训练模式下用当前的均值和方差做标准化
        X_hat = (X - mean) / torch.sqrt(var + eps)
        # 更新移动平均的均值和方差
        moving_mean = momentum * moving_mean + (1.0 - momentum) * mean
        moving_var = momentum * moving_var + (1.0 - momentum) * var
    Y = gamma * X_hat + beta  # 拉伸和偏移
    return Y, moving_mean, moving_var

In [ ]:
class BatchNorm(nn.Module):
    def __init__(self, num_features, num_dims):
        super(BatchNorm, self).__init__()
        if num_dims == 2:
            shape = (1, num_features) #全连接层输出神经元
        else:
            shape = (1, num_features, 1, 1)  #通道数
        # 参与求梯度和迭代的拉伸和偏移参数，分别初始化成0和1
        self.gamma = nn.Parameter(torch.ones(shape))
        self.beta = nn.Parameter(torch.zeros(shape))
        # 不参与求梯度和迭代的变量，全在内存上初始化成0
        self.moving_mean = torch.zeros(shape)
        self.moving_var = torch.zeros(shape)

    def forward(self, X):
        # 如果X不在内存上，将moving_mean和moving_var复制到X所在显存上
        if self.moving_mean.device != X.device:
            self.moving_mean = self.moving_mean.to(X.device)
            self.moving_var = self.moving_var.to(X.device)
        # 保存更新过的moving_mean和moving_var, Module实例的traning属性默认为true, 调用.eval()后设成false
        Y, self.moving_mean, self.moving_var = batch_norm(self.training, 
            X, self.gamma, self.beta, self.moving_mean,
            self.moving_var, eps=1e-5, momentum=0.9)
        return Y

#### 在Lenet中的应用

In [ ]:
net = nn.Sequential(
            nn.Conv2d(1, 6, 5), # in_channels, out_channels, kernel_size
            BatchNorm(6, num_dims=4),
            nn.Sigmoid(),
            nn.MaxPool2d(2, 2), # kernel_size, stride
            nn.Conv2d(6, 16, 5),
            BatchNorm(16, num_dims=4),
            nn.Sigmoid(),
            nn.MaxPool2d(2, 2),
            d2l.FlattenLayer(),
            nn.Linear(16*4*4, 120),
            BatchNorm(120, num_dims=2),
            nn.Sigmoid(),
            nn.Linear(120, 84),
            BatchNorm(84, num_dims=2),
            nn.Sigmoid(),
            nn.Linear(84, 10)
        )
print(net)

#### 简洁实现

In [ ]:
net = nn.Sequential(
            nn.Conv2d(1, 6, 5), # in_channels, out_channels, kernel_size
            nn.BatchNorm2d(6),
            nn.Sigmoid(),
            nn.MaxPool2d(2, 2), # kernel_size, stride
            nn.Conv2d(6, 16, 5),
            nn.BatchNorm2d(16),
            nn.Sigmoid(),
            nn.MaxPool2d(2, 2),
            d2l.FlattenLayer(),
            nn.Linear(16*4*4, 120),
            nn.BatchNorm1d(120),
            nn.Sigmoid(),
            nn.Linear(120, 84),
            nn.BatchNorm1d(84),
            nn.Sigmoid(),
            nn.Linear(84, 10)
        )

optimizer = torch.optim.Adam(net.parameters(), lr=lr)
d2l.train_ch5(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs)

### 卷积神经网络的整体结构
通过组合卷积层，汇聚层，全连接层，卷积层用来识别图像里的空间模式，之后池化层用来降低卷积层对位置的敏感性,最后全连接层对物体进行分类。我们就可以获得以下一般的卷积神经网络结构：
<div align=center>

![Untitled/Untitled.png](Pics/convstruct.png)

</div>

不过网络结构并不是一成不变的，而是根据需要不断去修改其中卷积的操作，层级的安排等等。我们现在已经越来越少见到卷积神经网络中的全连接层了，汇聚层的比例也在不断减少。

## 经典的卷积神经网络
通过学习一些经典的神经网络，我们可以学习到其中设计网络的思想

### Lenet

Lenet的网络结构也遵循了我们之前提到的（卷积+激活+池化）*N+全连接的公式。
* 卷积层块由两个这样的基本单位重复堆叠构成。在卷积层块中，每个卷积层都使用 5×5 的窗口，并在输出上使用sigmoid激活函数。第一个卷积层输出通道数为6，第二个卷积层输出通道数则增加到16。
* 全连接层块含3个全连接层。它们的输出个数分别是120、84和10，其中10为输出的类别个数。
<div align=center>

![Untitled/Untitled.png](Pics/lenet.png)

</div>

### Alexnet
相较于Lenet，Alexnet是一个更加现代的卷积神经网络，其使用更深的网络来提升网络的参数/表征空间，并用一些技巧来控制模型的复杂度。主要有以下特征：
1. 8层变换，其中有5层卷积和2层全连接隐藏层，以及1个全连接输出层。
2. 将sigmoid激活函数改成了更加简单的ReLU激活函数。
3. 用Dropout来控制全连接层的模型复杂度。
4. 引入数据增强，如翻转、裁剪和颜色变化，从而进一步扩大数据集来缓解过拟合。

<div align=center>

![Image Name](Pics/ALEX.png)

</div>

除此之外，Alexnet还使用全卷积的方式来减少显存的占用，以往，我们每生成一张输出特征图就需要使用和输入相同channels的卷积核进行卷积，在群卷积中，每次只需要使用输入channels/K的卷积核进行运算，并把最后结果concate起来，就能获得和原来相同大小的输出。
群卷积有助于减小参数的数量，但是其限制了每个卷积核对于通道的访问，从而限制了特征的组合。下图阐述了群卷积的运行：
<div align=center>

![gconv.png](Pics/gconv.png)
</div>

In [ ]:
import time
import torch
from torch import nn, optim
import torchvision
import numpy as np
import sys
import os
import torch.nn.functional as F

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 96, 11, 4), # in_channels, out_channels, kernel_size, stride, padding
            nn.ReLU(),
            nn.MaxPool2d(3, 2), # kernel_size, stride
            # 减小卷积窗口，使用填充为2来使得输入与输出的高和宽一致，且增大输出通道数
            nn.Conv2d(96, 256, 5, 1, 2),
            nn.ReLU(),
            nn.MaxPool2d(3, 2),
            # 连续3个卷积层，且使用更小的卷积窗口。除了最后的卷积层外，进一步增大了输出通道数。
            # 前两个卷积层后不使用池化层来减小输入的高和宽
            nn.Conv2d(256, 384, 3, 1, 1),
            nn.ReLU(),
            nn.Conv2d(384, 384, 3, 1, 1),
            nn.ReLU(),
            nn.Conv2d(384, 256, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(3, 2)
        )
         # 这里全连接层的输出个数比LeNet中的大数倍。使用丢弃层来缓解过拟合
        self.fc = nn.Sequential(
            nn.Linear(256*5*5, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            #由于使用CPU镜像，精简网络，若为GPU镜像可添加该层
            #nn.Linear(4096, 4096),
            #nn.ReLU(),
            #nn.Dropout(0.5),

            # 输出层。由于这里使用Fashion-MNIST，所以用类别数为10，而非论文中的1000
            nn.Linear(4096, 10),
        )

    def forward(self, img):

        feature = self.conv(img)
        output = self.fc(feature.view(img.shape[0], -1))
        return output

### VGG
使用多个重复的模块进行叠加的网络结构。
* Block:数个相同的填充为1、窗口形状为 3×3 的卷积层,接上一个步幅为2、窗口形状为 2×2 的最大池化层。
* 卷积层保持输入的高和宽不变，而池化层则对其减半。
<div align=center>

![Image Name](Pics/VGG.png)

</div>

In [ ]:
## VGG11 的简单实现

def vgg_block(num_convs, in_channels, out_channels): #卷积层个数，输入通道数，输出通道数
    blk = []
    for i in range(num_convs):
        if i == 0:
            blk.append(nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1))
        else:
            blk.append(nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1))
        blk.append(nn.ReLU())
    blk.append(nn.MaxPool2d(kernel_size=2, stride=2)) # 这里会使宽高减半
    return nn.Sequential(*blk)

conv_arch = ((1, 1, 64), (1, 64, 128), (2, 128, 256), (2, 256, 512), (2, 512, 512))
# 经过5个vgg_block, 宽高会减半5次, 变成 224/32 = 7
fc_features = 512 * 7 * 7 # c * w * h
fc_hidden_units = 4096 # 任意

def vgg(conv_arch, fc_features, fc_hidden_units=4096):
    net = nn.Sequential()
    # 卷积层部分
    for i, (num_convs, in_channels, out_channels) in enumerate(conv_arch):
        # 每经过一个vgg_block都会使宽高减半
        net.add_module("vgg_block_" + str(i+1), vgg_block(num_convs, in_channels, out_channels))
    # 全连接层部分
    net.add_module("fc", nn.Sequential(d2l.FlattenLayer(),
                                 nn.Linear(fc_features, fc_hidden_units),
                                 nn.ReLU(),
                                 nn.Dropout(0.5),
                                 nn.Linear(fc_hidden_units, fc_hidden_units),
                                 nn.ReLU(),
                                 nn.Dropout(0.5),
                                 nn.Linear(fc_hidden_units, 10)
                                ))
    return net


In [ ]:
net = vgg(conv_arch, fc_features, fc_hidden_units)
X = torch.rand(1, 1, 224, 224)

# named_children获取一级子模块及其名字(named_modules会返回所有子模块,包括子模块的子模块)
for name, blk in net.named_children(): 
    X = blk(X)
    print(name, 'output shape: ', X.shape)

### 网络中的网络
__LeNet、AlexNet和VGG__：先以由卷积层构成的模块充分抽取 空间特征，再以由全连接层构成的模块来输出分类结果。

__NiN__：
串联多个由卷积层和“全连接”层构成的小⽹络来构建⼀个深层⽹络。
⽤了输出通道数等于标签类别数的NiN块，然后使⽤全局平均池化层对每个通道中所有元素求平均并直接⽤于分类。

![Image Name](Pics/NIN.png)

1×1卷积核作用:

- 放缩通道数：通过控制卷积核的数量达到通道数的放缩。
- 增加非线性。1×1卷积核的卷积过程相当于全连接层的计算过程，并且还加入了非线性激活函数，从而可以增加网络的非线性。
- 充当全连接层的作用，减少卷积层到全连接层之间的显式数据转换（4维张量的flatten操作）



In [ ]:
## NIN 网络的简单实现

def nin_block(in_channels, out_channels, kernel_size, stride, padding):
    blk = nn.Sequential(nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding),
                        nn.ReLU(),
                        nn.Conv2d(out_channels, out_channels, kernel_size=1),
                        nn.ReLU(),
                        nn.Conv2d(out_channels, out_channels, kernel_size=1),
                        nn.ReLU())
    return blk

class GlobalAvgPool2d(nn.Module):
    # 全局平均池化层可通过将池化窗口形状设置成输入的高和宽实现
    def __init__(self):
        super(GlobalAvgPool2d, self).__init__()
    def forward(self, x):
        return F.avg_pool2d(x, kernel_size=x.size()[2:])

net = nn.Sequential(
    nin_block(1, 96, kernel_size=11, stride=4, padding=0),
    nn.MaxPool2d(kernel_size=3, stride=2),
    nin_block(96, 256, kernel_size=5, stride=1, padding=2),
    nn.MaxPool2d(kernel_size=3, stride=2),
    nin_block(256, 384, kernel_size=3, stride=1, padding=1),
    nn.MaxPool2d(kernel_size=3, stride=2), 
    nn.Dropout(0.5),
    # 标签类别数是10
    nin_block(384, 10, kernel_size=3, stride=1, padding=1),
    GlobalAvgPool2d(), 
    # 将四维的输出转成二维的输出，其形状为(批量大小, 10)
    d2l.FlattenLayer())


In [ ]:
X = torch.rand(1, 1, 224, 224)
for name, blk in net.named_children(): 
    X = blk(X)
    print(name, 'output shape: ', X.shape)

# GoogLenet
通过串联卷积模块和 Inception 模块来搭建网络结构。


__Inception 模块：__ 
- 采用多个不同分支来提取不同细粒度的特征，从左到右，提取的特征越来越抽象；
- 同时，为了能使得最后的特征能够拼接，需要对不同的分支采用不同的 padding 等操作，保证输出的长和宽不变；
- 其中为了减少计算参数，在中间两个分支都采用了 1\*1 的卷积层来减少输出通道，从而减少卷积核的通道数。 
- 最后，通过控制不同分支的输出通道数来控制生成的 Inception 模块的复杂度

![googlenet.png](Pics/googlenet.png)


In [ ]:
class Inception(nn.Module):
    # c1 - c4为每条线路里的层的输出通道数
    def __init__(self, in_c, c1, c2, c3, c4):
        super(Inception, self).__init__()
        # 线路1，单1 x 1卷积层
        self.p1_1 = nn.Conv2d(in_c, c1, kernel_size=1)
        # 线路2，1 x 1卷积层后接3 x 3卷积层
        self.p2_1 = nn.Conv2d(in_c, c2[0], kernel_size=1)
        self.p2_2 = nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1)
        # 线路3，1 x 1卷积层后接5 x 5卷积层
        self.p3_1 = nn.Conv2d(in_c, c3[0], kernel_size=1)
        self.p3_2 = nn.Conv2d(c3[0], c3[1], kernel_size=5, padding=2)
        # 线路4，3 x 3最大池化层后接1 x 1卷积层
        self.p4_1 = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        self.p4_2 = nn.Conv2d(in_c, c4, kernel_size=1)

    def forward(self, x):
        p1 = F.relu(self.p1_1(x))
        p2 = F.relu(self.p2_2(F.relu(self.p2_1(x))))
        p3 = F.relu(self.p3_2(F.relu(self.p3_1(x))))
        p4 = F.relu(self.p4_2(self.p4_1(x)))
        return torch.cat((p1, p2, p3, p4), dim=1)  # 在通道维上连结输出

#### GoogLenet 的完整结构 
![img](Pics/googlenet_full.png)

In [ ]:
b1 = nn.Sequential(nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3),
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

b2 = nn.Sequential(nn.Conv2d(64, 64, kernel_size=1),
                   nn.Conv2d(64, 192, kernel_size=3, padding=1),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

b3 = nn.Sequential(Inception(192, 64, (96, 128), (16, 32), 32),
                   Inception(256, 128, (128, 192), (32, 96), 64),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

b4 = nn.Sequential(Inception(480, 192, (96, 208), (16, 48), 64),
                   Inception(512, 160, (112, 224), (24, 64), 64),
                   Inception(512, 128, (128, 256), (24, 64), 64),
                   Inception(512, 112, (144, 288), (32, 64), 64),
                   Inception(528, 256, (160, 320), (32, 128), 128),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

b5 = nn.Sequential(Inception(832, 256, (160, 320), (32, 128), 128),
                   Inception(832, 384, (192, 384), (48, 128), 128),
                   d2l.GlobalAvgPool2d())

net = nn.Sequential(b1, b2, b3, b4, b5, 
                    d2l.FlattenLayer(), nn.Linear(1024, 10))

net = nn.Sequential(b1, b2, b3, b4, b5, d2l.FlattenLayer(), nn.Linear(1024, 10))

X = torch.rand(1, 1, 96, 96)

for blk in net.children(): 
    X = blk(X)
    print('output shape: ', X.shape)

#batchsize=128
batch_size = 16
# 如出现“out of memory”的报错信息，可减小batch_size或resize
#train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=96)

lr, num_epochs = 0.001, 5
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
d2l.train_ch5(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs)

### Resnet

通常来说，深层神经网路的表示函数能包含浅层神经网络的所有表示函数，因此，我们希望能根据输入数据，在更多的可能性中寻找合适的映射函数。但是，在使用深层神经网络时，我们往往会遇到过拟合问题，即神经网络没有拟合数据的一般规律，而是拟合了这一组数据中的噪声，造成神经网络的泛化能力差。因此，为了解决能更好地应用深层网络，Resnet 提供了一种跳接的连接方式，其示意图如下：

![img](Pics/residual.png)

可以发现，其通过将输入恒等加入到输出，促使后层网络能直接获取输入的信息，变相说来，这种跳接给网络提供了一种选择: 是否需要采用这个新加入的神经层，其也就使得网络能根据输入数据在复杂和简单的网络结构中进行良好的切换，实验结构表明，使用了 Resnet 跳接的网络可以增加很多层而不容易产生过拟合。

除此之外， Residual 模块提供了在恒等变化的基础上捕捉变化的能力，促使网络表达更加细腻。


In [ ]:
class Residual(nn.Module):  
    #可以设定输出通道数、是否使用额外的1x1卷积层来修改通道数以及卷积层的步幅。
    # 由于 resnet 是将结果相加，因此输入输出的形状需要一致，因此我们还需要用1*1卷积来统一可能变化的通道数
    def __init__(self, in_channels, out_channels, use_1x1conv=False, stride=1):
        super(Residual, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, stride=stride)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
        if use_1x1conv:
            self.conv3 = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride)
        else:
            self.conv3 = None
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)

    def forward(self, X):
        Y = F.relu(self.bn1(self.conv1(X)))
        Y = self.bn2(self.conv2(Y))
        if self.conv3:
            X = self.conv3(X)
        return F.relu(Y + X)

### ResNet模型
- 卷积(64,7x7,3)  
- 批量一体化  
- 最大池化(3x3,2)  

- 残差块x4 (通过步幅为2的残差块在每个模块之间减小高和宽)

- 全局平均池化

- 全连接

In [ ]:
net = nn.Sequential(
        nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3),
        nn.BatchNorm2d(64), 
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

def resnet_block(in_channels, out_channels, num_residuals, first_block=False):
    if first_block:
        assert in_channels == out_channels # 第一个模块的通道数同输入通道数一致
    blk = []
    for i in range(num_residuals):
        if i == 0 and not first_block:
            blk.append(Residual(in_channels, out_channels, use_1x1conv=True, stride=2))
        else:
            blk.append(Residual(out_channels, out_channels))
    return nn.Sequential(*blk)

net.add_module("resnet_block1", resnet_block(64, 64, 2, first_block=True))
net.add_module("resnet_block2", resnet_block(64, 128, 2))
net.add_module("resnet_block3", resnet_block(128, 256, 2))
net.add_module("resnet_block4", resnet_block(256, 512, 2))
net.add_module("global_avg_pool", d2l.GlobalAvgPool2d()) # GlobalAvgPool2d的输出: (Batch, 512, 1, 1)
net.add_module("fc", nn.Sequential(d2l.FlattenLayer(), nn.Linear(512, 10))) 

In [ ]:
X = torch.rand((1, 1, 224, 224))
for name, layer in net.named_children():
    X = layer(X)
    print(name, ' output shape:\t', X.shape)

### DenseNet
densenet 通过将跳接之后的连接过程从相加修改为 concatenate，形成了一个更激进的密集连接机制：即互相连接所有的层，即每个层都会接受其前面所有层作为其额外的输入。这样一来，后层可以直接访问前面任意层特征图，促进了信息综合。

![img](Pics/densenet.png)

![img](Pics/densenetjump.png)

具体说来，其优势如下：
1. DenseNet提升了梯度的反向传播，使得网络更容易训练。由于每层可以直达最后的误差信号，实现了隐式的“deep supervision”；
2. 参数更小且计算更高效，其采用较小的growth rate，实现每个层所独有的特征图是比较小的；同时采用过渡层也减少了参数
3. 由于特征复用，最后的分类器使用了低级特征，最后决策也就有机会采用多个尺度的特征

# 优化
优化是为了在所有模型表示空间中利用参数迭代方法（优化方法）找到损失函数最小的模型表示。

## 深度学习优化：
深度学习模型的优化是为了优化针对深度模型进行的优化，在这一部分我们会主要描述深度学习模型的优化和传统优化的差异，以及基于梯度的优化方法在深度学习优化中遇到的问题。

### 深度学习优化和普通优化的差异
一般的优化我们只需要在已知数据上进行优化，即针对训练集进行优化，但是对于深度学习模型而言，我们需要通过在训练数据上的优化获得在未知数据上的损失函数最小值。而训练损失和测试损失并不一直拥有一致的变化，其中一种可能的变化如下：

![img](Pics/testtrainerror.svg)

为了解决这类问题，我们需要对网络建立训练集，验证集，同时保证训练集和验证集本身能反映真实数据的分布，这样我们就能利用验证集上的表现选取良好的模型，正如之前模型选择中提到的那样。
### 基于梯度的优化方法的挑战
除了上述训练误差和测试误差不匹配的问题，我们用基于梯度的方法优化深度学习模型还会遇到以下几点问题

#### 局部最优点

#### 鞍点

#### 梯度消失



首先我们阐述一下基于梯度的算法的普遍形式，这里基于梯度的方法是一种基于线搜索的优化方法。在搜索优化过程中，我们希望能对于每个自变量寻找到一个方向，再乘以一个步长，使得我们在将自变量不断往这个方向前进的过程，能促使目标函数不断减小。而基于梯度的下降算法就认为这个方向是梯度的反方向，即迭代公式如下：
$$
f(x)_{new} = f(x - \eta f^{\prime}(x))
$$

这个结论的正确性是明显的。下面我们用两种不同的方法来证明：

**证明：沿梯度反方向移动自变量可以减小函数值**

1. 图形的方法：
我们的自变量的变化范围都是在其周围的局部极小领域变化的。

我们知道梯度为函数在各个方向的导数，而函数导数大于0代表函数在自变量增加的方向函数值增加，导数小于0则反之；因此某一维度导数大于0时将自变量在减小，某一维度导数小于0时函数增大，就能减小函数值。说明梯度的反方向就是函数值减小的方向。其实，梯度的方向还是函数值下降最快的方向。

2. 基于泰勒展开：
首先，泰勒展开公式如下：
$$
f(x+\epsilon)=f(x)+\epsilon f^{\prime}(x)+\mathcal{O}\left(\epsilon^{2}\right)
$$

对于梯度下降算法，我们沿梯度方向的移动量 $\eta f^{\prime}(x)$，并假设这一个移动量是极小量，因此可以看成是$\epsilon$，并进行泰勒展开：

$$
f\left(x-\eta f^{\prime}(x)\right)=f(x)-\eta f^{\prime 2}(x)+\mathcal{O}\left(\eta^{2} f^{\prime 2}(x)\right)
$$

只要$\eta$较小，我们就能保证（因为左边是右边的高阶无穷小量）：
$$
 \mathcal{O}\left(\eta^{2} f^{\prime 2}(x)\right) \lesssim \eta f^{\prime 2}(x)
$$
从而我们就证明了
$$
f\left(x-\eta f^{\prime}(x)\right) \lesssim f(x)
$$


$$
x \leftarrow x-\eta f^{\prime}(x)
$$
那么，这样说来，是不是基于梯度下降的方法就能万无一失地最快地找到最小点了呢，其实并不，因为梯度方向是函数下降最快地方向是针对$x$的一个局部极小领域来说的，因此我们不能保证乘以一个步长之后会到什么位置，同时我们也不能看到局部极小领域之外的范围函数值是如何变化来决定我们的下降方向，这一点可以说是一个极大的限制了。

下面我们从三个方面阐述基于梯度优化方法的三个挑战

#### 局部最优点
局部最优点是基于梯度的算法的一大难点，因为局部最优时，梯度为0；因此，直接根据梯度下降算法，我们将停滞不前，而看不到前方有全局最优等着我们，这就是梯度下降算法的局部视野导致的。不过，在深度学习模型优化平面上，这种情形其实并不常见，因为局部极小需要我们的每一个自变量维度都是一个局部极小的状态，但是由于深度学习模型中自变量的维度实在太多，因此每个自变量都正好是局部极小的概率十分的小。下图展示了一种局部极小的情况：

In [ ]:
def f(x):
    return x * np.cos(np.pi * x)

![img](Pics/localmin.svg)

#### 鞍点
正如上述提到的局部极小存在极少的原因，这个原因却导致了在深度学习优化平面中鞍点的存在的普遍性。鞍点的定义就是在某些维度上优化点是局部极小，其他自变量维度上优化点是局部极大的情况。

其数学判定形式就是函数的 Hesse 矩阵有正有负，Hesse 矩阵是函数的在对自变量的二阶偏导数，其有正有负也就代表了鞍点的局部极小和局部极大的不确定性。

$$
A=\left[\begin{array}{cccc}{\frac{\partial^{2} f}{\partial x_{1}^{2}}} & {\frac{\partial^{2} f}{\partial x_{1} \partial x_{2}}} & {\cdots} & {\frac{\partial^{2} f}{\partial x_{1} \partial x_{n}}} \\ {\frac{\partial^{2} f}{\partial x_{2} \partial x_{1}}} & {\frac{\partial^{2} f}{\partial x_{2}^{2}}} & {\cdots} & {\frac{\partial^{2} f}{\partial x_{2} \partial x_{n}}} \\ {\vdots} & {\vdots} & {\ddots} & {\vdots} \\ {\frac{\partial^{2} f}{\partial x_{n} \partial x_{1}}} & {\frac{\partial^{2} f}{\partial x_{n} \partial x_{2}}} & {\cdots} & {\frac{\partial^{2} f}{\partial x_{n}^{2}}}\end{array}\right]
$$


鞍点的一种三维的表示如下：
![img](Pics/saddle.svg)

当碰到鞍点，可以说单纯的梯度下降算法已经完全失效，因为无论是局部极小还是局部极大，梯度都为 0，那么自变量也就不会更新了。为了解决鞍点问题，我们就需要在前进方向上添加一点扰动：

- 我们可以在梯度方向上增加一点随机量（这样带来的坏处是在全局最优点时还是有局部震荡）
- 或者我们可以将前进方向不仅由当前梯度决定，还可以由经验梯度带来参考等。


#### 梯度消失
梯度消失也是很好理解的，只要梯度越小，我们的前进速度就会越慢，那么优化也就会放缓，这是我们不希望的。不过我们可以通过调整学习率来弥补梯度过小/大的情况。后续我们会介绍两种方法来自适应调整梯度来保证梯度能较合适。

![img](Pics/vanishgradient.svg)

## 凸性 （Convexity）
凸性是描述一个聚合或者函数是否是凸的。

- 对于集合来说，凸集合是任意两点的连线均在集合中；
- 对于函数来说，凸函数就是函数上任意两点的连线上的点要大于相同自变量范围的对应函数值，或者就如 Jensen 不等式所说，函数值的期望大于期望的函数值

在优化学科中，很大一部分都在研究针对凸函数的优化，虽然深度模型的优化平面并不满足凸性，但是如果将领域缩小到一个较小范围，我们可以将使用凸函数进行近似，并且凸函数优化中也有很多思想可以借鉴，这一节我们主要介绍凸函数的性质和凸集合。

### 凸集合
下面分别阐述了凸集和非凸集的几种表现形式，其中第一行的后两个为凸集；同时第二行想说明的是凸集的交集也是凸集；第三行想说明凸集的并集就不是凸集了

![Image Name](https://cdn.kesci.com/upload/image/q5p1yhqzm8.png?imageView2/0/w/640/h/640)
![Image Name](https://cdn.kesci.com/upload/image/q5p1xz9gvz.png?imageView2/0/w/640/h/640)
![Image Name](https://cdn.kesci.com/upload/image/q5p1yue9fu.png?imageView2/0/w/320/h/320)



### 凸函数的判定式
正如之前所说：凸函数就是函数上任意两点的连线上的点要大于相同自变量范围的对应函数值，也就是以连线的端点组成的线性函数会在对应自变量线性组合对应的函数值的上方，这代表了凸函数一定只会有一个全局最小值而没有局部最小值，其用不等式的表示如下：


$$
\lambda f(x)+(1-\lambda) f\left(x^{\prime}\right) \geq f\left(\lambda x+(1-\lambda) x^{\prime}\right)
$$

#### Jensen 不等式，多维凸函数的判定式

当自变量有很多维度时，我们用下面的不等式表示。特别地，对于凸函数来说，任意在定义域内的自变量都可以使得函数的期望值大于期望的函数值
$$
\sum_{i} \alpha_{i} f\left(x_{i}\right) \geq f\left(\sum_{i} \alpha_{i} x_{i}\right) \text { and } E_{x}[f(x)] \geq f\left(E_{x}[x]\right)
$$

几种凸函数和非凸函数的曲线展示如下：


![img](Pics/convexfun.svg)

### 凸函数性质
1. 无局部极小值
2. 与凸集的关系
3. 二阶条件

####  1. 无局部最小值

证明：假设存在 $x \in X$ 是局部最小值，则存在全局最小值 $x' \in X$, 使得 $f(x) > f(x')$, 则对 $\lambda \in(0,1]$:

$$
f(x)>\lambda f(x^{\prime})+(1-\lambda) f(x) \geq f(\lambda x^{\prime}+(1-\lambda) x) = f(x + \lambda (x^{\prime}- x)) 
$$
上式与其为局部极小值矛盾（假设 $x^{\prime}- x$ 为极小量）


#### 2.与凸集的关系

通过下面的公式截取部分凸函数对应的定义域我们就可以得到凸集。

对于凸函数 $f(x)$，定义集合 $S_{b}:=\{x | x \in X \text { and } f(x) \leq b\}$，则集合 $S_b$ 为凸集

通过平面截取的定义域，能保证平面在截取的自变量范围对应的函数部分都小于 b，而由凸函数的性质，这部分是线性变化的，也就是满足凸集的定义。

证明：对于点 $x,x' \in S_b$, 有 $f\left(\lambda x+(1-\lambda) x^{\prime}\right) \leq \lambda f(x)+(1-\lambda) f\left(x^{\prime}\right) \leq b$, 故 $\lambda x+(1-\lambda) x^{\prime} \in S_{b}$

通过截取 $f(x, y)=0.5 x^{2}+\cos (2 \pi y)$ 的定义域，即便函数值都小于 b，我们也不能得到凸集，因为原函数并不是一个凸函数，示意图如下：

![img](Pics/convexfunset.svg)

#### 3.凸函数与二阶导数

$f^{''}(x) \ge 0 \Longleftrightarrow f(x)$ 是凸函数

**必要性 ($\Leftarrow$):**

对于凸函数：

$$
\frac{1}{2} f(x+\epsilon)+\frac{1}{2} f(x-\epsilon) \geq f\left(\frac{x+\epsilon}{2}+\frac{x-\epsilon}{2}\right)=f(x)
$$

故:

$$
f^{\prime \prime}(x)=\lim _{\varepsilon \rightarrow 0} \frac{\frac{f(x+\epsilon) - f(x)}{\epsilon}-\frac{f(x) - f(x-\epsilon)}{\epsilon}}{\epsilon}
$$


$$
f^{\prime \prime}(x)=\lim _{\varepsilon \rightarrow 0} \frac{f(x+\epsilon)+f(x-\epsilon)-2 f(x)}{\epsilon^{2}} \geq 0
$$


**充分性 ($\Rightarrow$):**

令 $a < x < b$ 为 $f(x)$ 上的三个点，由拉格朗日中值定理(

如果函数 $f(x)$ 满足：

- 在闭区间 $[a,b]$ 上连续;
- 在开区间 $[a,b]$ 内可微分;

则存在 $x \in [a ,b]$ 使得函数中 $f(a)$, $f(b)$ 两点的连线的斜率和 $f^{\prime}(x) $ 相同):

$$
\begin{array}{l}{f(x)-f(a)=(x-a) f^{\prime}(\alpha) \text { for some } \alpha \in[a, x] \text { and }} \\ {f(b)-f(x)=(b-x) f^{\prime}(\beta) \text { for some } \beta \in[x, b]}\end{array}
$$


根据单调性，有 $f^{\prime}(\beta) \geq f^{\prime}(\alpha)$, 故:

$$
\begin{aligned} f(b)-f(a) &=f(b)-f(x)+f(x)-f(a) \\ &=(b-x) f^{\prime}(\beta)+(x-a) f^{\prime}(\alpha) \\ & \geq(b-a) f^{\prime}(\alpha) \end{aligned}
$$

## 优化算法
这一章我们会介绍几种基于梯度的优化算法，从应用于一般优化问题的梯度下降算法出发，我们发现确定学习步长是一个重要的问题，即便在二阶方法-牛顿法中也是如此；接下来我们再介绍应用在深度学习模型中的随机梯度下降法和小批量随机梯度下降法。

### 最陡梯度下降法
正如我们之前介绍过的，梯度下降法使用$f(x)_{new} = f(x - \eta f{^\prime}(x) )$ 来更新自变量来获得更小的函数值；

我们定义损失函数如下：
对于有 $n$ 个样本对训练数据集，设 $f_i(x)$ 是第 $i$ 个样本的损失函数, 则目标函数为:

$$
f(\mathbf{x})=\frac{1}{n} \sum_{i=1}^{n} f_{i}(\mathbf{x})
$$

其梯度为:

$$
\nabla f(\mathbf{x})=\frac{1}{n} \sum_{i=1}^{n} \nabla f_{i}(\mathbf{x})
$$

其中需要注意的是，$f{^\prime}(x)$ 是通过获取全部数据的的梯度求得，因此用该梯度的一次更新的时间复杂度为 $\mathcal{O}(n)$

#### 一维情况
我们分几种情况探讨梯度下降法的使用情况。 下面，我们对于函数 $f(x) = x^2 $使用梯度下降法进行求解

##### 不同学习率

In [ ]:
def f(x):
    return x**2  # Objective function

def gradf(x):
    return 2 * x  # Its derivative

def gd(eta):
    x = 10
    results = [x]
    for i in range(10):
        x -= eta * gradf(x)
        results.append(x)
    print('epoch 10, x:', x)
    return results

res = gd(0.2)  # learning rate is 0.2

![img](Pics/x2.svg)

当我们设置学习率为 0.05 时，我们发现迭代变慢了
![img](Pics/x2small.svg)

当我们设置学习率为 1.1 时，我们发现迭代的函数值发散了

![img](Pics/x2big.svg)

可以发现，对于凸函数，梯度下降法我们需要较好的学习率才可以获得较好的迭代结果和迭代速度，其中学习率的设置可以参考这样一条规则，即学习率可以设置在 $[1/L, 2/L]$之间，当 $\eta = L$时有最快的下降速度。其中 L 为 Hesse 矩阵的特征值

我们了解了对于凸函数的情况，那么对于非凸函数表现如何呢？

##### 非凸函数上的表现
$$
f(x) = x\cos cx
$$

In [ ]:
c = 0.15 * np.pi

def f(x):
    return x * np.cos(c * x)

def gradf(x):
    return np.cos(c * x) - c * x * np.sin(c * x)


![img](Pics/x2nonconvex.svg)

可见其很容易收敛到局部极小点上

### 总结
- 梯度下降法对于学习率的设置较敏感，可以根据 $ \eta \in [1/L,2/L]$ 来设置
- 梯度下降法很容易陷入在局部最优

## 多维梯度下降


$$
\nabla f(\mathbf{x})=\left[\frac{\partial f(\mathbf{x})}{\partial x_{1}}, \frac{\partial f(\mathbf{x})}{\partial x_{2}}, \dots, \frac{\partial f(\mathbf{x})}{\partial x_{d}}\right]^{\top}
$$

$$
f(\mathbf{x}+\epsilon)=f(\mathbf{x})+\epsilon^{\top} \nabla f(\mathbf{x})+\mathcal{O}\left(\|\epsilon\|^{2}\right)
$$

$$
\mathbf{x} \leftarrow \mathbf{x}-\eta \nabla f(\mathbf{x})
$$

In [ ]:
def train_2d(trainer, steps=20):
    x1, x2 = -5, -2
    results = [(x1, x2)]
    for i in range(steps):
        x1, x2 = trainer(x1, x2)
        results.append((x1, x2))
    print('epoch %d, x1 %f, x2 %f' % (i + 1, x1, x2))
    return results

def show_trace_2d(f, results):
    d2l.plt.plot(*zip(*results), '-o', color='#ff7f0e')
    x1, x2 = np.meshgrid(np.arange(-5.5, 1.0, 0.1), np.arange(-3.0, 1.0, 0.1))
    d2l.plt.contour(x1, x2, f(x1, x2), colors='#1f77b4')
    d2l.plt.xlabel('x1')
    d2l.plt.ylabel('x2')


$$
f(x) = x_1^2 + 2x_2^2
$$


In [ ]:
eta = 0.1

def f_2d(x1, x2):  # 目标函数
    return x1 ** 2 + 2 * x2 ** 2

def gd_2d(x1, x2):
    return (x1 - eta * 2 * x1, x2 - eta * 4 * x2)

show_trace_2d(f_2d, train_2d(gd_2d))

![IMG](Pics/multi.svg)

## 自适应方法
之前我们发现设置学习率对基于梯度的优化方法影响很大，下面我们来看看一些希望自适应获得学习率的方法

### 牛顿法
牛顿法希望通过解析方式获得最优解，通过设置导数等于 0 求得解析解如下所示：

在 $x + \epsilon$ 处泰勒展开：

$$
f(\mathbf{x}+\epsilon)=f(\mathbf{x})+\epsilon^{\top} \nabla f(\mathbf{x})+\frac{1}{2} \epsilon^{\top} \nabla \nabla^{\top} f(\mathbf{x}) \epsilon+\mathcal{O}\left(\|\epsilon\|^{3}\right)
$$

最小值点处满足: $\nabla f(\mathbf{x})=0$, 即我们希望 $\nabla f(\mathbf{x} + \epsilon)=0$, 对上式关于 $\epsilon$ 求导，忽略高阶无穷小，有：

$$
\nabla f(\mathbf{x})+\boldsymbol{H}_{f} \boldsymbol{\epsilon}=0 \text { and hence } \epsilon=-\boldsymbol{H}_{f}^{-1} \nabla f(\mathbf{x})
$$

也许你发现通过解析解，我们可以只用迭代一步就找到全局最优。但是，泰勒展开仅当 $\epsilon$ 为极小量时成立，因此，牛顿法实际上并不能一步获得全局最优。

In [ ]:
c = 0.5

def f(x):
    return np.cosh(c * x)  # Objective

def gradf(x):
    return c * np.sinh(c * x)  # Derivative

def hessf(x):
    return c**2 * np.cosh(c * x)  # Hessian

# Hide learning rate for now
def newton(eta=1):
    x = 10
    results = [x]
    for i in range(10):
        x -= eta * gradf(x) / hessf(x)
        results.append(x)
    print('epoch 10, x:', x)
    return results

show_trace(newton())

In [ ]:
c = 0.15 * np.pi

def f(x):
    return x * np.cos(c * x)

def gradf(x):
    return np.cos(c * x) - c * x * np.sin(c * x)

def hessf(x):
    return - 2 * c * np.sin(c * x) - x * c**2 * np.cos(c * x)

show_trace(newton())

牛顿法遇到局部极小也会产生发散的问题，需要降低学习率来实现。
也许你发现牛顿法的初衷就是为了解决学习率设置的问题，但是到最后遇到非凸函数其还是不得不自行改变学习率，因此牛顿法其实主要还是适应于凸函数中不用调整学习率的情况。

### 总结
- 实施牛顿法需要计算 d\*d 的 Hesse 矩阵的逆（d 为自变量的个数），计算量非常大，但是对于最优解在局部领域内的情况可以一步到达最优解。
- 牛顿法对于凸函数可以省去设置学习率的过程，获得快速收敛的最优解。
- 牛顿法对于非凸函数依旧需要设置学习率来满足 Wolfe 条件，Wolfe 条件分别是最快下降条件和曲率条件，其保证了下降的速率。详细说明可以见[这里](https://www.jiqizhixin.com/graph/technologies/4761e6c4-223c-431f-9110-8b184068a885)


### 收敛性分析

只考虑在函数为凸函数, 且最小值点上 $f''(x^*) > 0$ 时的收敛速度：

令 $x_k$ 为第 $k$ 次迭代后 $x$ 的值， $e_{k}:=x_{k}-x^{*}$ 表示 $x_k$ 到最小值点 $x^{*}$ 的距离，由 $f'(x^{*}) = 0$:

$$
0=f^{\prime}\left(x_{k}-e_{k}\right)=f^{\prime}\left(x_{k}\right)-e_{k} f^{\prime \prime}\left(x_{k}\right)+\frac{1}{2} e_{k}^{2} f^{\prime \prime \prime}\left(\xi_{k}\right) \text{for some } \xi_{k} \in\left[x_{k}-e_{k}, x_{k}\right]
$$

两边除以 $f''(x_k)$, 有：

$$
e_{k}-f^{\prime}\left(x_{k}\right) / f^{\prime \prime}\left(x_{k}\right)=\frac{1}{2} e_{k}^{2} f^{\prime \prime \prime}\left(\xi_{k}\right) / f^{\prime \prime}\left(x_{k}\right)
$$

代入更新方程 $x_{k+1} = x_{k} - f^{\prime}\left(x_{k}\right) / f^{\prime \prime}\left(x_{k}\right)$, 得到：

$$
x_k - x^{*} - f^{\prime}\left(x_{k}\right) / f^{\prime \prime}\left(x_{k}\right) =\frac{1}{2} e_{k}^{2} f^{\prime \prime \prime}\left(\xi_{k}\right) / f^{\prime \prime}\left(x_{k}\right)
$$


$$
x_{k+1} - x^{*} = e_{k+1} = \frac{1}{2} e_{k}^{2} f^{\prime \prime \prime}\left(\xi_{k}\right) / f^{\prime \prime}\left(x_{k}\right)
$$

当 $\frac{1}{2} f^{\prime \prime \prime}\left(\xi_{k}\right) / f^{\prime \prime}\left(x_{k}\right) \leq c$ 时，有:

$$
e_{k+1} \leq c e_{k}^{2}
$$



结论： 牛顿法迭代一次可以使得 $x_k+1$ 到最小值点 $x^{*}$ 的距离 小于等于 $x_k$ 到最小值点 $x^{*}$ 的距离的平方的 c 倍

### 预处理 （Heissan阵辅助梯度下降）
牛顿法的另一个应用是，对于优化平面比较扁平的时候，我们可以使用 Heissan 矩阵中的对角线元素来规范前进的步长。


$$
\mathbf{x} \leftarrow \mathbf{x}-\eta \operatorname{diag}\left(H_{f}\right)^{-1} \nabla \mathbf{x}
$$



### 共轭梯度法
共轭梯度法是介于最速下降法与牛顿法之间的一个方法，它仅需一阶导数信息，但克服了最速下降法收敛慢的缺点，又避免了牛顿法需要存储和计算Hesse矩阵并求逆的缺点。 共轭梯度法的基本思想是把共轭性与最速下降法相结合，利用已知点处的梯度构造一组共轭方向，并沿这组方向进行搜索，求出目标函数的极小点。

共轭梯度的设计思想是通过构造共轭的梯度搜索方向，在仅需一阶导数信息的基础上，加快梯度下降法的迭代速度。其收敛速率能保证在 n 步之内，其中 n 为自变量的维度。


![IMG](Pics/Conjugate_gradient_illustration.svg)


更多参考[Wikipedia](https://en.wikipedia.org/wiki/Conjugate_gradient_method)


## 随机梯度下降


## 随机梯度下降参数更新

对于有 $n$ 个样本对训练数据集，设 $f_i(x)$ 是第 $i$ 个样本的损失函数, 则目标函数为:

$$
f(\mathbf{x})=\frac{1}{n} \sum_{i=1}^{n} f_{i}(\mathbf{x})
$$

其梯度为:

$$
\nabla f(\mathbf{x})=\frac{1}{n} \sum_{i=1}^{n} \nabla f_{i}(\mathbf{x})
$$

使用该梯度的一次更新的时间复杂度为 $\mathcal{O}(n)$

随机梯度下降更新公式 $\mathcal{O}(1)$:

$$
\mathbf{x} \leftarrow \mathbf{x}-\eta \nabla f_{i}(\mathbf{x})
$$

且有：

$$
\mathbb{E}_{i} \nabla f_{i}(\mathbf{x})=\frac{1}{n} \sum_{i=1}^{n} \nabla f_{i}(\mathbf{x})=\nabla f(\mathbf{x})
$$


从上述推导可以看到，梯度下降方法的更新参数时间复杂度过高，因此我们在深度神经网络中采用随机梯度下降方法，其每次使用一个样本的梯度用来更新，这样可行的原因是通过把单个样本的梯度当成是对所有样本的梯度的平均估计

e.g.

$$
f(x_1, x_2) = x_1^2 + 2 x_2^2
$$


In [7]:
def f(x1, x2):
    return x1 ** 2 + 2 * x2 ** 2  # Objective

def gradf(x1, x2):
    return (2 * x1, 4 * x2)  # Gradient

def sgd(x1, x2):  # Simulate noisy gradient
    global lr  # Learning rate scheduler
    (g1, g2) = gradf(x1, x2)  # Compute gradient
    (g1, g2) = (g1 + np.random.normal(0.1), g2 + np.random.normal(0.1))
    eta_t = eta * lr()  # Learning rate at time t
    return (x1 - eta_t * g1, x2 - eta_t * g2)  # Update variables

eta = 0.1
lr = (lambda: 1)  # Constant learning rate
show_trace_2d(f, train_2d(sgd, steps=50))


NameError: name 'show_trace_2d' is not defined

可以看到，随机梯度下降法对于凸函数依旧可以找到最小点，但是其因为样本估计梯度中的噪声在最优值附近抖动，同时由于没有利用上并行计算，其速度较慢。

### 动态学习率
为了解决上述提到的由于样本估计梯度中的噪声在最优值附近抖动的问题，我们可以设置学习率策略（学习率先大后小），限制优化后期梯度中的噪声的影响。


$$
\begin{array}{ll}{\eta(t)=\eta_{i} \text { if } t_{i} \leq t \leq t_{i+1}} & {\text { piecewise constant }} \\ {\eta(t)=\eta_{0} \cdot e^{-\lambda t}} & {\text { exponential }} \\ {\eta(t)=\eta_{0} \cdot(\beta t+1)^{-\alpha}} & {\text { polynomial }}\end{array}
$$


In [ ]:
def exponential():
    global ctr
    ctr += 1
    return math.exp(-0.1 * ctr)

ctr = 1
lr = exponential  # Set up learning rate
show_trace_2d(f, train_2d(sgd, steps=1000))

In [ ]:
def polynomial():
    global ctr
    ctr += 1
    return (1 + 0.1 * ctr)**(-0.5)

ctr = 1
lr = polynomial  # Set up learning rate
show_trace_2d(f, train_2d(sgd, steps=50))

通过改变学习率变化策略，随机梯度下降法的后期抖动减少不少

# 小批量随机梯度下降

## [读取数据](https://archive.ics.uci.edu/ml/datasets/Airfoil+Self-Noise)

In [ ]:
def get_data_ch7():  # 本函数已保存在d2lzh_pytorch包中方便以后使用
    data = np.genfromtxt('/home/kesci/input/airfoil4755/airfoil_self_noise.dat', delimiter='\t')
    data = (data - data.mean(axis=0)) / data.std(axis=0) # 标准化
    return torch.tensor(data[:1500, :-1], dtype=torch.float32), \
           torch.tensor(data[:1500, -1], dtype=torch.float32) # 前1500个样本(每个样本5个特征)

features, labels = get_data_ch7()
features.shape

In [ ]:
import pandas as pd
df = pd.read_csv('/home/kesci/input/airfoil4755/airfoil_self_noise.dat', delimiter='\t', header=None)
df.head(10)

## 从零开始实现

In [ ]:
def sgd(params, states, hyperparams):
    for p in params:
        p.data -= hyperparams['lr'] * p.grad.data

In [ ]:
# 本函数已保存在d2lzh_pytorch包中方便以后使用
def train_ch7(optimizer_fn, states, hyperparams, features, labels,
              batch_size=10, num_epochs=2):
    # 初始化模型
    net, loss = d2l.linreg, d2l.squared_loss

    w = torch.nn.Parameter(torch.tensor(np.random.normal(0, 0.01, size=(features.shape[1], 1)), dtype=torch.float32),
                           requires_grad=True)
    b = torch.nn.Parameter(torch.zeros(1, dtype=torch.float32), requires_grad=True)

    def eval_loss():
        return loss(net(features, w, b), labels).mean().item()

    ls = [eval_loss()]
    data_iter = torch.utils.data.DataLoader(
        torch.utils.data.TensorDataset(features, labels), batch_size, shuffle=True)

    for _ in range(num_epochs):
        start = time.time()
        for batch_i, (X, y) in enumerate(data_iter):
            l = loss(net(X, w, b), y).mean()  # 使用平均损失

            # 梯度清零
            if w.grad is not None:
                w.grad.data.zero_()
                b.grad.data.zero_()

            l.backward()
            optimizer_fn([w, b], states, hyperparams)  # 迭代模型参数
            if (batch_i + 1) * batch_size % 100 == 0:
                ls.append(eval_loss())  # 每100个样本记录下当前训练误差
    # 打印结果和作图
    print('loss: %f, %f sec per epoch' % (ls[-1], time.time() - start))
    d2l.set_figsize()
    d2l.plt.plot(np.linspace(0, num_epochs, len(ls)), ls)
    d2l.plt.xlabel('epoch')
    d2l.plt.ylabel('loss')

In [ ]:
def train_sgd(lr, batch_size, num_epochs=2):
    train_ch7(sgd, None, {'lr': lr}, features, labels, batch_size, num_epochs)

对比

In [ ]:
train_sgd(1, 1500, 6)   # 梯度下降

In [ ]:
train_sgd(0.005, 1) # 随机梯度下降

In [ ]:
train_sgd(0.05, 10)  # 小批量梯度下降

## 简洁实现

In [ ]:
# 本函数与原书不同的是这里第一个参数优化器函数而不是优化器的名字
# 例如: optimizer_fn=torch.optim.SGD, optimizer_hyperparams={"lr": 0.05}
def train_pytorch_ch7(optimizer_fn, optimizer_hyperparams, features, labels,
                    batch_size=10, num_epochs=2):
    # 初始化模型
    net = nn.Sequential(
        nn.Linear(features.shape[-1], 1)
    )
    loss = nn.MSELoss()
    optimizer = optimizer_fn(net.parameters(), **optimizer_hyperparams)

    def eval_loss():
        return loss(net(features).view(-1), labels).item() / 2

    ls = [eval_loss()]
    data_iter = torch.utils.data.DataLoader(
        torch.utils.data.TensorDataset(features, labels), batch_size, shuffle=True)

    for _ in range(num_epochs):
        start = time.time()
        for batch_i, (X, y) in enumerate(data_iter):
            # 除以2是为了和train_ch7保持一致, 因为squared_loss中除了2
            l = loss(net(X).view(-1), y) / 2

            optimizer.zero_grad()
            l.backward()
            optimizer.step()
            if (batch_i + 1) * batch_size % 100 == 0:
                ls.append(eval_loss())
    # 打印结果和作图
    print('loss: %f, %f sec per epoch' % (ls[-1], time.time() - start))
    d2l.set_figsize()
    d2l.plt.plot(np.linspace(0, num_epochs, len(ls)), ls)
    d2l.plt.xlabel('epoch')
    d2l.plt.ylabel('loss')

In [ ]:
train_pytorch_ch7(optim.SGD, {"lr": 0.05}, features, labels, 10)


### 总结
1. 随机梯度下降法：
    - 计算复杂度小：为了减小梯度下降法的计算复杂度设置，具有更新计算量小 $\mathcal{O}(1)$ 的特点
    - 计算效率低：没有利用上并行计算的能力，计算相比小批量梯度下降法更加慢
    - 优化后期抖动：由于单样本估计的梯度具有噪声，导致其容易在优化后期抖动，这一点可以通过动态设置学习率来解决

2. 小批量梯度下降：
    - 计算复杂度小：同$\mathcal{O}(1)$
    - 计算效率较高：增加 Batch Size 利用并行计算
    - 后期抖动较小

## Momentum

这一节我们通过介绍两种方法来解决优化平面 ill-conditioned 的问题，并就这个问题引出现有的很多优化算法

In [ ]:
## An ill-conditioned Problem

Condition Number of Hessian Matrix:

$$
 cond_{H} = \frac{\lambda_{max}}{\lambda_{min}}
$$

where $\lambda_{max}, \lambda_{min}$ is the maximum amd minimum eignvalue of Hessian matrix.

让我们考虑一个输入和输出分别为二维向量$\boldsymbol{x} = [x_1, x_2]^\top$和标量的目标函数:


$$
 f(\boldsymbol{x})=0.1x_1^2+2x_2^2
$$



$$
 cond_{H} = \frac{4}{0.2} = 20 \quad \rightarrow \quad \text{ill-conditioned}
$$


## Maximum Learning Rate
+ For $f(x)$, according to convex optimizaiton conclusions, we need step size $\eta$.
+ To guarantee the convergence, we need to have $\eta$ .

## Supp: Preconditioning

在二阶优化中，我们使用Hessian matrix的逆矩阵(或者pseudo inverse)来左乘梯度向量 $i.e. \Delta_{x} = H^{-1}\mathbf{g}$，这样的做法称为precondition，相当于将 $H$ 映射为一个单位矩阵，拥有分布均匀的Spectrum，也即我们去优化的等价标函数的Hessian matrix为良好的identity matrix。


与[Section 11.4](https://d2l.ai/chapter_optimization/sgd.html#sec-sgd)一节中不同，这里将$x_1^2$系数从$1$减小到了$0.1$。下面实现基于这个目标函数的梯度下降，并演示使用学习率为$0.4$时自变量的迭代轨迹。

可以看到，同一位置上，目标函数在竖直方向（$x_2$轴方向）比在水平方向（$x_1$轴方向）的斜率的绝对值更大。因此，给定学习率，梯度下降迭代自变量时会使自变量在竖直方向比在水平方向移动幅度更大。那么，我们需要一个较小的学习率从而避免自变量在竖直方向上越过目标函数最优解。然而，这会造成自变量在水平方向上朝最优解移动变慢。

下面我们试着将学习率调得稍大一点，此时自变量在竖直方向不断越过最优解并逐渐发散。

### Solution to ill-condition
+ __Preconditioning gradient vector__: applied in Adam, RMSProp, AdaGrad, Adelta, KFC, Natural gradient and other secord-order optimization algorithms.
+ __Averaging history gradient__: like momentum, which allows larger learning rates to accelerate convergence; applied in Adam, RMSProp, SGD momentum.

In [ ]:
eta = 0.6
d2l.show_trace_2d(f_2d, d2l.train_2d(gd_2d))

## Momentum Algorithm

动量法的提出是为了解决梯度下降的上述问题。设时间步 $t$ 的自变量为 $\boldsymbol{x}_t$，学习率为 $\eta_t$。
在时间步 $t=0$，动量法创建速度变量 $\boldsymbol{m}_0$，并将其元素初始化成 0。在时间步 $t>0$，动量法对每次迭代的步骤做如下修改：


$$
\begin{aligned}
\boldsymbol{m}_t &\leftarrow \beta \boldsymbol{m}_{t-1} + \eta_t \boldsymbol{g}_t, \\
\boldsymbol{x}_t &\leftarrow \boldsymbol{x}_{t-1} - \boldsymbol{m}_t,
\end{aligned}
$$

Another version:

$$
\begin{aligned}
\boldsymbol{m}_t &\leftarrow \beta \boldsymbol{m}_{t-1} + (1-\beta) \boldsymbol{g}_t, \\
\boldsymbol{x}_t &\leftarrow \boldsymbol{x}_{t-1} - \alpha_t \boldsymbol{m}_t,
\end{aligned}
$$

$$
\alpha_t = \frac{\eta_t}{1-\beta}
$$

其中，动量超参数 $\beta$满足 $0 \leq \beta < 1$。当 $\beta=0$ 时，动量法等价于小批量随机梯度下降。

在解释动量法的数学原理前，让我们先从实验中观察梯度下降在使用动量法后的迭代轨迹。

In [ ]:
def momentum_2d(x1, x2, v1, v2):
    v1 = beta * v1 + eta * 0.2 * x1
    v2 = beta * v2 + eta * 4 * x2
    return x1 - v1, x2 - v2, v1, v2

eta, beta = 0.4, 0.5
d2l.show_trace_2d(f_2d, d2l.train_2d(momentum_2d))

可以看到使用较小的学习率 $\eta=0.4$ 和动量超参数 $\beta=0.5$ 时，动量法在竖直方向上的移动更加平滑，且在水平方向上更快逼近最优解。下面使用较大的学习率 $\eta=0.6$，此时自变量也不再发散。

In [ ]:
eta = 0.6
d2l.show_trace_2d(f_2d, d2l.train_2d(momentum_2d))

### Exponential Moving Average

为了从数学上理解动量法，让我们先解释一下指数加权移动平均（exponential moving average）。给定超参数 $0 \leq \beta < 1$，当前时间步 $t$ 的变量 $y_t$ 是上一时间步 $t-1$ 的变量 $y_{t-1}$ 和当前时间步另一变量 $x_t$ 的线性组合：

$$
y_t = \beta y_{t-1} + (1-\beta) x_t.
$$

我们可以对 $y_t$ 展开：

$$
\begin{aligned}
y_t  &= (1-\beta) x_t + \beta y_{t-1}\\
         &= (1-\beta)x_t + (1-\beta) \cdot \beta x_{t-1} + \beta^2y_{t-2}\\
         &= (1-\beta)x_t + (1-\beta) \cdot \beta x_{t-1} + (1-\beta) \cdot \beta^2x_{t-2} + \beta^3y_{t-3}\\
         &= (1-\beta) \sum_{i=0}^{t} \beta^{i}x_{t-i}
\end{aligned}
$$

$$
(1-\beta)\sum_{i=0}^{t} \beta^{i} = \frac{1-\beta^{t}}{1-\beta} (1-\beta) = (1-\beta^{t})
$$

### Supp
Approximate Average of $\frac{1}{1-\beta}$ Steps

令 $n = 1/(1-\beta)$，那么 $\left(1-1/n\right)^n = \beta^{1/(1-\beta)}$。因为

$$
 \lim_{n \rightarrow \infty}  \left(1-\frac{1}{n}\right)^n = \exp(-1) \approx 0.3679,
$$

所以当 $\beta \rightarrow 1$时，$\beta^{1/(1-\beta)}=\exp(-1)$，如 $0.95^{20} \approx \exp(-1)$。如果把 $\exp(-1)$ 当作一个比较小的数，我们可以在近似中忽略所有含 $\beta^{1/(1-\beta)}$ 和比 $\beta^{1/(1-\beta)}$ 更高阶的系数的项。例如，当 $\beta=0.95$ 时，

$$
y_t \approx 0.05 \sum_{i=0}^{19} 0.95^i x_{t-i}.
$$

因此，在实际中，我们常常将 $y_t$ 看作是对最近 $1/(1-\beta)$ 个时间步的 $x_t$ 值的加权平均。例如，当 $\gamma = 0.95$ 时，$y_t$ 可以被看作对最近20个时间步的 $x_t$ 值的加权平均；当 $\beta = 0.9$ 时，$y_t$ 可以看作是对最近10个时间步的 $x_t$ 值的加权平均。而且，离当前时间步 $t$ 越近的 $x_t$ 值获得的权重越大（越接近1）。


### 由指数加权移动平均理解动量法

现在，我们对动量法的速度变量做变形：

$$
\boldsymbol{m}_t \leftarrow \beta \boldsymbol{m}_{t-1} + (1 - \beta) \left(\frac{\eta_t}{1 - \beta} \boldsymbol{g}_t\right).
$$

Another version:

$$
\boldsymbol{m}_t \leftarrow \beta \boldsymbol{m}_{t-1} + (1 - \beta) \boldsymbol{g}_t.
$$


$$
\begin{aligned}
\boldsymbol{x}_t &\leftarrow \boldsymbol{x}_{t-1} - \alpha_t \boldsymbol{m}_t,
\end{aligned}
$$


$$
\alpha_t = \frac{\eta_t}{1-\beta}
$$


由指数加权移动平均的形式可得，速度变量 $\boldsymbol{v}_t$ 实际上对序列 $\{\eta_{t-i}\boldsymbol{g}_{t-i} /(1-\beta):i=0,\ldots,1/(1-\beta)-1\}$ 做了指数加权移动平均。换句话说，相比于小批量随机梯度下降，动量法在每个时间步的自变量更新量近似于将前者对应的最近 $1/(1-\beta)$ 个时间步的更新量做了指数加权移动平均后再除以 $1-\beta$。所以，在动量法中，自变量在各个方向上的移动幅度不仅取决当前梯度，还取决于过去的各个梯度在各个方向上是否一致。在本节之前示例的优化问题中，所有梯度在水平方向上为正（向右），而在竖直方向上时正（向上）时负（向下）。这样，我们就可以使用较大的学习率，从而使自变量向最优解更快移动。


## Implement

相对于小批量随机梯度下降，动量法需要对每一个自变量维护一个同它一样形状的速度变量，且超参数里多了动量超参数。实现中，我们将速度变量用更广义的状态变量`states`表示。

In [ ]:
def get_data_ch7():
    data = np.genfromtxt('/home/kesci/input/airfoil4755/airfoil_self_noise.dat', delimiter='\t')
    data = (data - data.mean(axis=0)) / data.std(axis=0)
    return torch.tensor(data[:1500, :-1], dtype=torch.float32), \
        torch.tensor(data[:1500, -1], dtype=torch.float32)

features, labels = get_data_ch7()

def init_momentum_states():
    v_w = torch.zeros((features.shape[1], 1), dtype=torch.float32)
    v_b = torch.zeros(1, dtype=torch.float32)
    return (v_w, v_b)

def sgd_momentum(params, states, hyperparams):
    for p, v in zip(params, states):
        v.data = hyperparams['momentum'] * v.data + hyperparams['lr'] * p.grad.data
        p.data -= v.data

我们先将动量超参数`momentum`设0.5

In [ ]:
d2l.train_ch7(sgd_momentum, init_momentum_states(),
              {'lr': 0.02, 'momentum': 0.5}, features, labels)

将动量超参数`momentum`增大到0.9

In [ ]:
d2l.train_ch7(sgd_momentum, init_momentum_states(),
              {'lr': 0.02, 'momentum': 0.9}, features, labels)

可见目标函数值在后期迭代过程中的变化不够平滑。直觉上，10倍小批量梯度比2倍小批量梯度大了5倍，我们可以试着将学习率减小到原来的1/5。此时目标函数值在下降了一段时间后变化更加平滑。

In [ ]:
d2l.train_ch7(sgd_momentum, init_momentum_states(),
              {'lr': 0.004, 'momentum': 0.9}, features, labels)

## Pytorch Class

在Pytorch中，```torch.optim.SGD```已实现了Momentum。

In [ ]:
d2l.train_pytorch_ch7(torch.optim.SGD, {'lr': 0.004, 'momentum': 0.9},
                    features, labels)

# 11.7 AdaGrad

在之前介绍过的优化算法中，目标函数自变量的每一个元素在相同时间步都使用同一个学习率来自我迭代。举个例子，假设目标函数为$f$，自变量为一个二维向量$[x_1, x_2]^\top$，该向量中每一个元素在迭代时都使用相同的学习率。例如，在学习率为$\eta$的梯度下降中，元素$x_1$和$x_2$都使用相同的学习率$\eta$来自我迭代：


$$

x_1 \leftarrow x_1 - \eta \frac{\partial{f}}{\partial{x_1}}, \quad
x_2 \leftarrow x_2 - \eta \frac{\partial{f}}{\partial{x_2}}.

$$


在[“动量法”](./momentum.ipynb)一节里我们看到当$x_1$和$x_2$的梯度值有较大差别时，需要选择足够小的学习率使得自变量在梯度值较大的维度上不发散。但这样会导致自变量在梯度值较小的维度上迭代过慢。动量法依赖指数加权移动平均使得自变量的更新方向更加一致，从而降低发散的可能。本节我们介绍AdaGrad算法，它根据自变量在每个维度的梯度值的大小来调整各个维度上的学习率，从而避免统一的学习率难以适应所有维度的问题 [1]。


## Algorithm

AdaGrad算法会使用一个小批量随机梯度$\boldsymbol{g}_t$按元素平方的累加变量$\boldsymbol{s}_t$。在时间步0，AdaGrad将$\boldsymbol{s}_0$中每个元素初始化为0。在时间步$t$，首先将小批量随机梯度$\boldsymbol{g}_t$按元素平方后累加到变量$\boldsymbol{s}_t$：


$$
\boldsymbol{s}_t \leftarrow \boldsymbol{s}_{t-1} + \boldsymbol{g}_t \odot \boldsymbol{g}_t,
$$


其中$\odot$是按元素相乘。接着，我们将目标函数自变量中每个元素的学习率通过按元素运算重新调整一下：


$$
\boldsymbol{x}_t \leftarrow \boldsymbol{x}_{t-1} - \frac{\eta}{\sqrt{\boldsymbol{s}_t + \epsilon}} \odot \boldsymbol{g}_t,
$$


其中$\eta$是学习率，$\epsilon$是为了维持数值稳定性而添加的常数，如$10^{-6}$。这里开方、除法和乘法的运算都是按元素运算的。这些按元素运算使得目标函数自变量中每个元素都分别拥有自己的学习率。

## Feature

需要强调的是，小批量随机梯度按元素平方的累加变量$\boldsymbol{s}_t$出现在学习率的分母项中。因此，如果目标函数有关自变量中某个元素的偏导数一直都较大，那么该元素的学习率将下降较快；反之，如果目标函数有关自变量中某个元素的偏导数一直都较小，那么该元素的学习率将下降较慢。然而，由于$\boldsymbol{s}_t$一直在累加按元素平方的梯度，自变量中每个元素的学习率在迭代过程中一直在降低（或不变）。所以，当学习率在迭代早期降得较快且当前解依然不佳时，AdaGrad算法在迭代后期由于学习率过小，可能较难找到一个有用的解。

下面我们仍然以目标函数$f(\boldsymbol{x})=0.1x_1^2+2x_2^2$为例观察AdaGrad算法对自变量的迭代轨迹。我们实现AdaGrad算法并使用和上一节实验中相同的学习率0.4。可以看到，自变量的迭代轨迹较平滑。但由于$\boldsymbol{s}_t$的累加效果使学习率不断衰减，自变量在迭代后期的移动幅度较小。

In [ ]:
%matplotlib inline
import math
import torch
import sys
sys.path.append("/home/kesci/input")
import d2lzh1981 as d2l

def adagrad_2d(x1, x2, s1, s2):
    g1, g2, eps = 0.2 * x1, 4 * x2, 1e-6  # 前两项为自变量梯度
    s1 += g1 ** 2
    s2 += g2 ** 2
    x1 -= eta / math.sqrt(s1 + eps) * g1
    x2 -= eta / math.sqrt(s2 + eps) * g2
    return x1, x2, s1, s2

def f_2d(x1, x2):
    return 0.1 * x1 ** 2 + 2 * x2 ** 2

eta = 0.4
d2l.show_trace_2d(f_2d, d2l.train_2d(adagrad_2d))

下面将学习率增大到2。可以看到自变量更为迅速地逼近了最优解。

In [ ]:
eta = 2
d2l.show_trace_2d(f_2d, d2l.train_2d(adagrad_2d))

## Implement

同动量法一样，AdaGrad算法需要对每个自变量维护同它一样形状的状态变量。我们根据AdaGrad算法中的公式实现该算法。

In [ ]:
def get_data_ch7():
    data = np.genfromtxt('/home/kesci/input/airfoil4755/airfoil_self_noise.dat', delimiter='\t')
    data = (data - data.mean(axis=0)) / data.std(axis=0)
    return torch.tensor(data[:1500, :-1], dtype=torch.float32), \
        torch.tensor(data[:1500, -1], dtype=torch.float32)

features, labels = get_data_ch7()

def init_adagrad_states():
    s_w = torch.zeros((features.shape[1], 1), dtype=torch.float32)
    s_b = torch.zeros(1, dtype=torch.float32)
    return (s_w, s_b)

def adagrad(params, states, hyperparams):
    eps = 1e-6
    for p, s in zip(params, states):
        s.data += (p.grad.data**2)
        p.data -= hyperparams['lr'] * p.grad.data / torch.sqrt(s + eps)

使用更大的学习率来训练模型。

In [ ]:
d2l.train_ch7(adagrad, init_adagrad_states(), {'lr': 0.1}, features, labels)

## Pytorch Class

通过名称为“adagrad”的`Trainer`实例，我们便可使用Pytorch提供的AdaGrad算法来训练模型。

In [ ]:
d2l.train_pytorch_ch7(torch.optim.Adagrad, {'lr': 0.1}, features, labels)

# 11.8 RMSProp

我们在[“AdaGrad算法”](adagrad.ipynb)一节中提到，因为调整学习率时分母上的变量$\boldsymbol{s}_t$一直在累加按元素平方的小批量随机梯度，所以目标函数自变量每个元素的学习率在迭代过程中一直在降低（或不变）。因此，当学习率在迭代早期降得较快且当前解依然不佳时，AdaGrad算法在迭代后期由于学习率过小，可能较难找到一个有用的解。为了解决这一问题，RMSProp算法对AdaGrad算法做了修改。该算法源自Coursera上的一门课程，即“机器学习的神经网络”。

## Algorithm

我们在[“动量法”](momentum.ipynb)一节里介绍过指数加权移动平均。不同于AdaGrad算法里状态变量$\boldsymbol{s}_t$是截至时间步$t$所有小批量随机梯度$\boldsymbol{g}_t$按元素平方和，RMSProp算法将这些梯度按元素平方做指数加权移动平均。具体来说，给定超参数$0 \leq \gamma 0$计算


$$
\boldsymbol{v}_t \leftarrow \beta \boldsymbol{v}_{t-1} + (1 - \beta) \boldsymbol{g}_t \odot \boldsymbol{g}_t.
$$


和AdaGrad算法一样，RMSProp算法将目标函数自变量中每个元素的学习率通过按元素运算重新调整，然后更新自变量


$$
\boldsymbol{x}_t \leftarrow \boldsymbol{x}_{t-1} - \frac{\alpha}{\sqrt{\boldsymbol{v}_t + \epsilon}} \odot \boldsymbol{g}_t,
$$


其中$\eta$是学习率，$\epsilon$是为了维持数值稳定性而添加的常数，如$10^{-6}$。因为RMSProp算法的状态变量$\boldsymbol{s}_t$是对平方项$\boldsymbol{g}_t \odot \boldsymbol{g}_t$的指数加权移动平均，所以可以看作是最近$1/(1-\beta)$个时间步的小批量随机梯度平方项的加权平均。如此一来，自变量每个元素的学习率在迭代过程中就不再一直降低（或不变）。

照例，让我们先观察RMSProp算法对目标函数$f(\boldsymbol{x})=0.1x_1^2+2x_2^2$中自变量的迭代轨迹。回忆在[“AdaGrad算法”](adagrad.ipynb)一节使用的学习率为0.4的AdaGrad算法，自变量在迭代后期的移动幅度较小。但在同样的学习率下，RMSProp算法可以更快逼近最优解。

In [ ]:
%matplotlib inline
import math
import torch
import sys
sys.path.append("/home/kesci/input")
import d2lzh1981 as d2l

def rmsprop_2d(x1, x2, s1, s2):
    g1, g2, eps = 0.2 * x1, 4 * x2, 1e-6
    s1 = beta * s1 + (1 - beta) * g1 ** 2
    s2 = beta * s2 + (1 - beta) * g2 ** 2
    x1 -= alpha / math.sqrt(s1 + eps) * g1
    x2 -= alpha / math.sqrt(s2 + eps) * g2
    return x1, x2, s1, s2

def f_2d(x1, x2):
    return 0.1 * x1 ** 2 + 2 * x2 ** 2

alpha, beta = 0.4, 0.9
d2l.show_trace_2d(f_2d, d2l.train_2d(rmsprop_2d))

## Implement

接下来按照RMSProp算法中的公式实现该算法。

In [ ]:
def get_data_ch7():
    data = np.genfromtxt('/home/kesci/input/airfoil4755/airfoil_self_noise.dat', delimiter='\t')
    data = (data - data.mean(axis=0)) / data.std(axis=0)
    return torch.tensor(data[:1500, :-1], dtype=torch.float32), \
        torch.tensor(data[:1500, -1], dtype=torch.float32)

features, labels = get_data_ch7()

def init_rmsprop_states():
    s_w = torch.zeros((features.shape[1], 1), dtype=torch.float32)
    s_b = torch.zeros(1, dtype=torch.float32)
    return (s_w, s_b)

def rmsprop(params, states, hyperparams):
    gamma, eps = hyperparams['beta'], 1e-6
    for p, s in zip(params, states):
        s.data = gamma * s.data + (1 - gamma) * (p.grad.data)**2
        p.data -= hyperparams['lr'] * p.grad.data / torch.sqrt(s + eps)

我们将初始学习率设为0.01，并将超参数$\gamma$设为0.9。此时，变量$\boldsymbol{s}_t$可看作是最近$1/(1-0.9) = 10$个时间步的平方项$\boldsymbol{g}_t \odot \boldsymbol{g}_t$的加权平均。

In [ ]:
d2l.train_ch7(rmsprop, init_rmsprop_states(), {'lr': 0.01, 'beta': 0.9},
              features, labels)

## Pytorch Class

通过名称为“rmsprop”的`Trainer`实例，我们便可使用Gluon提供的RMSProp算法来训练模型。注意，超参数$\gamma$通过`gamma1`指定。

In [ ]:
d2l.train_pytorch_ch7(torch.optim.RMSprop, {'lr': 0.01, 'alpha': 0.9},
                    features, labels)

# 11.9 AdaDelta

除了RMSProp算法以外，另一个常用优化算法AdaDelta算法也针对AdaGrad算法在迭代后期可能较难找到有用解的问题做了改进 [1]。有意思的是，AdaDelta算法没有学习率这一超参数。

## Algorithm

AdaDelta算法也像RMSProp算法一样，使用了小批量随机梯度$\boldsymbol{g}_t$按元素平方的指数加权移动平均变量$\boldsymbol{s}_t$。在时间步0，它的所有元素被初始化为0。给定超参数$0 \leq \rho 0$，同RMSProp算法一样计算


$$
\boldsymbol{s}_t \leftarrow \rho \boldsymbol{s}_{t-1} + (1 - \rho) \boldsymbol{g}_t \odot \boldsymbol{g}_t.
$$


与RMSProp算法不同的是，AdaDelta算法还维护一个额外的状态变量$\Delta\boldsymbol{x}_t$，其元素同样在时间步0时被初始化为0。我们使用$\Delta\boldsymbol{x}_{t-1}$来计算自变量的变化量：


$$
 \boldsymbol{g}_t' \leftarrow \sqrt{\frac{\Delta\boldsymbol{x}_{t-1} + \epsilon}{\boldsymbol{s}_t + \epsilon}}   \odot \boldsymbol{g}_t,
$$


其中$\epsilon$是为了维持数值稳定性而添加的常数，如$10^{-5}$。接着更新自变量：


$$
\boldsymbol{x}_t \leftarrow \boldsymbol{x}_{t-1} - \boldsymbol{g}'_t.
$$


最后，我们使用$\Delta\boldsymbol{x}_t$来记录自变量变化量$\boldsymbol{g}'_t$按元素平方的指数加权移动平均：


$$
\Delta\boldsymbol{x}_t \leftarrow \rho \Delta\boldsymbol{x}_{t-1} + (1 - \rho) \boldsymbol{g}'_t \odot \boldsymbol{g}'_t.
$$


可以看到，如不考虑$\epsilon$的影响，AdaDelta算法与RMSProp算法的不同之处在于使用$\sqrt{\Delta\boldsymbol{x}_{t-1}}$来替代超参数$\eta$。


## Implement

AdaDelta算法需要对每个自变量维护两个状态变量，即$\boldsymbol{s}_t$和$\Delta\boldsymbol{x}_t$。我们按AdaDelta算法中的公式实现该算法。

In [ ]:
def init_adadelta_states():
    s_w, s_b = torch.zeros((features.shape[1], 1), dtype=torch.float32), torch.zeros(1, dtype=torch.float32)
    delta_w, delta_b = torch.zeros((features.shape[1], 1), dtype=torch.float32), torch.zeros(1, dtype=torch.float32)
    return ((s_w, delta_w), (s_b, delta_b))

def adadelta(params, states, hyperparams):
    rho, eps = hyperparams['rho'], 1e-5
    for p, (s, delta) in zip(params, states):
        s[:] = rho * s + (1 - rho) * (p.grad.data**2)
        g =  p.grad.data * torch.sqrt((delta + eps) / (s + eps))
        p.data -= g
        delta[:] = rho * delta + (1 - rho) * g * g

In [ ]:
d2l.train_ch7(adadelta, init_adadelta_states(), {'rho': 0.9}, features, labels)

## Pytorch Class

通过名称为“adadelta”的`Trainer`实例，我们便可使用pytorch提供的AdaDelta算法。它的超参数可以通过`rho`来指定。

In [ ]:
d2l.train_pytorch_ch7(torch.optim.Adadelta, {'rho': 0.9}, features, labels)

# 11.10 Adam

Adam算法在RMSProp算法基础上对小批量随机梯度也做了指数加权移动平均 [1]。下面我们来介绍这个算法。

## Algorithm

Adam算法使用了动量变量$\boldsymbol{m}_t$和RMSProp算法中小批量随机梯度按元素平方的指数加权移动平均变量$\boldsymbol{v}_t$，并在时间步0将它们中每个元素初始化为0。给定超参数$0 \leq \beta_1 < 1$（算法作者建议设为0.9），时间步$t$的动量变量$\boldsymbol{m}_t$即小批量随机梯度$\boldsymbol{g}_t$的指数加权移动平均：


$$
\boldsymbol{m}_t \leftarrow \beta_1 \boldsymbol{m}_{t-1} + (1 - \beta_1) \boldsymbol{g}_t.
$$


和RMSProp算法中一样，给定超参数$0 \leq \beta_2 < 1$（算法作者建议设为0.999），
将小批量随机梯度按元素平方后的项$\boldsymbol{g}_t \odot \boldsymbol{g}_t$做指数加权移动平均得到$\boldsymbol{v}_t$：


$$
\boldsymbol{v}_t \leftarrow \beta_2 \boldsymbol{v}_{t-1} + (1 - \beta_2) \boldsymbol{g}_t \odot \boldsymbol{g}_t.
$$


由于我们将$\boldsymbol{m}_0$和$\boldsymbol{s}_0$中的元素都初始化为0，
在时间步$t$我们得到$\boldsymbol{m}_t =  (1-\beta_1) \sum_{i=1}^t \beta_1^{t-i} \boldsymbol{g}_i$。将过去各时间步小批量随机梯度的权值相加，得到 $(1-\beta_1) \sum_{i=1}^t \beta_1^{t-i} = 1 - \beta_1^t$。需要注意的是，当$t$较小时，过去各时间步小批量随机梯度权值之和会较小。例如，当$\beta_1 = 0.9$时，$\boldsymbol{m}_1 = 0.1\boldsymbol{g}_1$。为了消除这样的影响，对于任意时间步$t$，我们可以将$\boldsymbol{m}_t$再除以$1 - \beta_1^t$，从而使过去各时间步小批量随机梯度权值之和为1。这也叫作偏差修正。在Adam算法中，我们对变量$\boldsymbol{m}_t$和$\boldsymbol{v}_t$均作偏差修正：


$$
\hat{\boldsymbol{m}}_t \leftarrow \frac{\boldsymbol{m}_t}{1 - \beta_1^t},
$$



$$
\hat{\boldsymbol{v}}_t \leftarrow \frac{\boldsymbol{v}_t}{1 - \beta_2^t}.
$$



接下来，Adam算法使用以上偏差修正后的变量$\hat{\boldsymbol{m}}_t$和$\hat{\boldsymbol{m}}_t$，将模型参数中每个元素的学习率通过按元素运算重新调整：


$$
\boldsymbol{g}_t' \leftarrow \frac{\eta \hat{\boldsymbol{m}}_t}{\sqrt{\hat{\boldsymbol{v}}_t} + \epsilon},
$$


其中$\eta$是学习率，$\epsilon$是为了维持数值稳定性而添加的常数，如$10^{-8}$。和AdaGrad算法、RMSProp算法以及AdaDelta算法一样，目标函数自变量中每个元素都分别拥有自己的学习率。最后，使用$\boldsymbol{g}_t'$迭代自变量：


$$
\boldsymbol{x}_t \leftarrow \boldsymbol{x}_{t-1} - \boldsymbol{g}_t'.
$$


## Implement

我们按照Adam算法中的公式实现该算法。其中时间步$t$通过`hyperparams`参数传入`adam`函数。

In [ ]:
%matplotlib inline
import torch
import sys
sys.path.append("/home/kesci/input")
import d2lzh1981 as d2l

def get_data_ch7():
    data = np.genfromtxt('/home/kesci/input/airfoil4755/airfoil_self_noise.dat', delimiter='\t')
    data = (data - data.mean(axis=0)) / data.std(axis=0)
    return torch.tensor(data[:1500, :-1], dtype=torch.float32), \
        torch.tensor(data[:1500, -1], dtype=torch.float32)

features, labels = get_data_ch7()

def init_adam_states():
    v_w, v_b = torch.zeros((features.shape[1], 1), dtype=torch.float32), torch.zeros(1, dtype=torch.float32)
    s_w, s_b = torch.zeros((features.shape[1], 1), dtype=torch.float32), torch.zeros(1, dtype=torch.float32)
    return ((v_w, s_w), (v_b, s_b))

def adam(params, states, hyperparams):
    beta1, beta2, eps = 0.9, 0.999, 1e-6
    for p, (v, s) in zip(params, states):
        v[:] = beta1 * v + (1 - beta1) * p.grad.data
        s[:] = beta2 * s + (1 - beta2) * p.grad.data**2
        v_bias_corr = v / (1 - beta1 ** hyperparams['t'])
        s_bias_corr = s / (1 - beta2 ** hyperparams['t'])
        p.data -= hyperparams['lr'] * v_bias_corr / (torch.sqrt(s_bias_corr) + eps)
    hyperparams['t'] += 1

In [ ]:
d2l.train_ch7(adam, init_adam_states(), {'lr': 0.01, 't': 1}, features, labels)

## Pytorch Class

In [ ]:
d2l.train_pytorch_ch7(torch.optim.Adam, {'lr': 0.01}, features, labels)
